In [1]:
# Python 标准库
import copy  # 用于复制对象（深拷贝）
import math  # 提供数学计算函数（如sqrt、log等）
import warnings  # 用于显示/管理警告信息
# 类型提示相关
from typing import List, Optional, Tuple, Union # 类型注解支持
# PyTorch 库
import torch
import torch.utils.checkpoint # 实现梯度检查点，节省显存
from torch import nn   # 包含神经网络层定义
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss # 常用损失函数
# Hugging Face Transformers 库
# 激活函数映射（字符串转函数）
from transformers.activations import ACT2FN
# 文本生成相关功能的 Mixin 基类
from transformers.generation import GenerationMixin
# 构建 attention mask 的工具函数（标准/Flash Attention）
from transformers.modeling_attn_mask_utils import (
    _prepare_4d_attention_mask,  # 一般 attention mask
    _prepare_4d_attention_mask_for_sdpa,  # SDPA 支持的 mask
    _prepare_4d_causal_attention_mask,  # causal attention mask
    _prepare_4d_causal_attention_mask_for_sdpa,  # causal mask for SDPA
)
# Flash Attention 支持检查及工具 判断是否支持左上角掩码（Flash Attention 限制）  判断 Flash Attention 是否可用
from transformers.modeling_flash_attention_utils import flash_attn_supports_top_left_mask, is_flash_attn_available
# 模型输出结构定义
from transformers.modeling_outputs import (
    BaseModelOutput,
    BaseModelOutputWithPastAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    Seq2SeqLMOutput,
    Seq2SeqModelOutput,
    Seq2SeqQuestionAnsweringModelOutput,
    Seq2SeqSequenceClassifierOutput,
)
from transformers.modeling_utils import PreTrainedModel # 模型基类
# 一些实用工具（文档注释、日志等）
from transformers.utils import (
    add_code_sample_docstrings,  # 添加代码示例文档
    add_end_docstrings,    # 添加文档结尾注释
    add_start_docstrings,  # 添加文档开头注释
    add_start_docstrings_to_model_forward,  # 给forward函数添加文档注释
    logging,   # 提供日志功能
    replace_return_docstrings, # 替换函数返回的文档描述
)
# BART 配置类（定义模型结构超参）
from transformers.models.bart.configuration_bart import BartConfig
# 若 Flash Attention 可用，则引入其 forward 实现
if is_flash_attn_available():
    from transformers.modeling_flash_attention_utils import _flash_attention_forward

2025-05-26 17:01:10.153717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748278870.341187      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748278870.397396      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
is_flash_attn_available()

False

In [3]:
from functools import lru_cache

In [ ]:
# 该函数 is_torch_npu_available 用于检测是否支持华为 NPU（Neural Processing Unit）
# 使用 functools.lru_cache 缓存结果，避免重复检测带来的性能开销；
# check_device 参数表示是否进一步检查设备可用性。
@lru_cache()
def is_torch_npu_available(check_device=False):
    # 若 PyTorch 本身不可用或系统未安装 torch_npu，直接返回 False。
    if not _torch_available or importlib.util.find_spec("torch_npu") is None:
        return False
    import torch
    import torch_npu  # 导入模块，确保其实际可用。
    # 若启用设备检查，则尝试访问 NPU 设备数量；
    # 如果设备不可用，会抛出异常，返回 False；
    # 否则返回 torch.npu.is_available()。
    if check_device:
        try:
            # 如果未找到 NPU，则会引发 RuntimeError
            _ = torch.npu.device_count()
            return torch.npu.is_available()
        except RuntimeError:
            return False
    # 在不检查设备的情况下，仅判断 torch.npu 属性存在且可用。
    return hasattr(torch, "npu") and torch.npu.is_available()

In [4]:
from transformers.utils import is_torch_npu_available

In [5]:
is_torch_npu_available()

False

In [ ]:
def is_flash_attn_available():
   # 如果包“flash-attn”可用，则可以本地使用flash-attention。
    if is_flash_attn_2_available():
        return True
    # Flash-Attention 可以在 Ascend NPU 上使用，无需安装 `flash-attn` 包
    if is_torch_npu_available():
        return True
    return False

In [6]:
from transformers.utils import is_torch_available

In [7]:
is_torch_available()

True

In [ ]:
# Flash Attention 必须在 CUDA（NVIDIA GPU）环境下才有效，这是因为它：
# 依赖 CUDA 内核：Flash Attention 是一种专门为高效 GPU 计算设计的算法，它使用自定义的 CUDA kernel 实现高性能的 attention 运算。
# 不能在 CPU 上运行：即使你安装了支持 Flash Attention 的版本，若没有 CUDA 设备，运行时也会自动 fallback 到普通 attention 实现。
# 对 GPU 架构有要求：目前 Flash Attention v1 和 v2 对硬件有不同要求，一般推荐使用 NVIDIA A100、L4、3090、4090 等较新显卡。
#  Flash Attention 无法在 TPU 上使用的原因：
# Flash Attention 是为 CUDA/GPU 优化的算法
# 它底层是用 CUDA kernel（NVIDIA专用） 写的，针对 NVIDIA GPU 的 warp/thread/block 架构深度优化。
# TPU 使用 XLA 编译器，不支持 CUDA
# TPU 上运行的是 XLA 编译的算子图，与 GPU 的 CUDA 编程模型完全不同，无法执行 CUDA kernel。
# Flash Attention 不具备通用实现
# 当前 Flash Attention（v1/v2）没有公开支持 XLA 或 TPU 专用版本，且其性能优势来自于精细的 CUDA 手工优化。
# ✅ TPU 上的替代方案：
# XLA 编译优化的原生 Attention：JAX/Flax 在 TPU 上的注意力计算可自动融合 + 编译优化；
# Memory-efficient Attention 实现（non-Flash）：如：
# flax.linen.attention.dot_product_attention（可控制是否使用 causal mask）
# xformers（虽然也是为 GPU 设计，但部分思路可迁移）

In [ ]:
def is_flash_attn_2_available():
    if not is_torch_available(): 
        return False
    if not _is_package_available("flash_attn"):
        return False
    # Let's add an extra check to see if cuda is available
    import torch
    if not (torch.cuda.is_available() or is_torch_mlu_available()):
        return False
    if torch.version.cuda:
        return version.parse(importlib.metadata.version("flash_attn")) >= version.parse("2.1.0")
    elif torch.version.hip:
        # TODO: Bump the requirement to 2.1.0 once released in https://github.com/ROCmSoftwarePlatform/flash-attention
        return version.parse(importlib.metadata.version("flash_attn")) >= version.parse("2.0.4")
    elif is_torch_mlu_available():
        return version.parse(importlib.metadata.version("flash_attn")) >= version.parse("2.3.3")
    else:
        return False

In [ ]:
!pip install flash_attn

In [9]:
from packaging import version
import importlib
version.parse(importlib.metadata.version("flash_attn"))

<Version('2.7.4.post1')>

In [10]:
from transformers.utils import is_flash_attn_2_available

In [11]:
is_flash_attn_2_available()

True

In [12]:
logger = logging.get_logger(__name__)

In [13]:
# 在官方文档、教程或示例代码中调用时，默认加载的BART基础预训练模型（"facebook/bart-base"）；
# 用于文档示例的预训练模型检查点名称，指向 Facebook 提供的基础版 BART 模型
_CHECKPOINT_FOR_DOC = "facebook/bart-base"
 # 用于文档示例的配置类名称，对应 BART 模型的配置接口
_CONFIG_FOR_DOC = "BartConfig"

In [14]:
# ✅ 用于说明 Base 模型（如 BartModel）在给定输入下的预期输出形状
# 用于 docstring 示例中模型 forward() 的预期输出形状验证
_EXPECTED_OUTPUT_SHAPE = [1, 8, 768]
# ✅ 用于文档中指定 Sequence Classification 模型的示例检查点（fine-tuned on SST-2）
_CHECKPOINT_FOR_SEQUENCE_CLASSIFICATION = "valhalla/bart-large-sst2"
# ✅ 示例中期望输出的损失值（如测试代码设定 label=gold，loss 应为 0）
_SEQ_CLASS_EXPECTED_LOSS = 0.0
# ✅ 说明当输入特定情感文本时，模型的输出标签应为 POSITIVE
_SEQ_CLASS_EXPECTED_OUTPUT = "'POSITIVE'"
# ✅ 用于文档中示例 QA 模型的检查点（fine-tuned on SQuAD v1）
_CHECKPOINT_FOR_QA = "valhalla/bart-large-finetuned-squadv1"
# ✅ 给定示例上下文/问题，计算损失的期望值，用于说明训练或推理效果
_QA_EXPECTED_LOSS = 0.59
# ✅ 对给定问题和上下文，期望的预测答案字符串，用于 doc 示例验证输出合理性
_QA_EXPECTED_OUTPUT = "' nice puppet'"

In [15]:
# 该函数用于 训练阶段解码器的输入预处理，将标签序列右移一位，插入起始 token，同时处理 -100 标记为 pad_token_id，
# 是 seq2seq 框架中训练时标准做法。
def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    # 创建与 input_ids 相同形状的零张量（保持 dtype 和 device），用于存储右移后的结果
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    # 将原 input_ids 除最后一个 token 外，右移一位，填入新张量中，从第1列开始
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    # 第0列设置为解码器起始 token（通常是 decoder_start_token_id，如 BART 中的 <s>）
    shifted_input_ids[:, 0] = decoder_start_token_id
    # 检查是否提供了 pad_token_id（因为后续要替换 -100）
    if pad_token_id is None:
        raise ValueError("self.model.config.pad_token_id has to be defined.")
    # 将所有位置中原本是 -100 的标记（如 label ignore 标记）替换为 pad_token_id，保证模型输入合法
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)
    return shifted_input_ids

In [16]:
# 该模块实现了可学习的位置嵌入（positional embeddings），用于为输入序列中的每个位置编码唯一的位置信息。
# 这种方式区别于固定的正弦位置编码，能通过训练自动优化位置向量，提升模型效果。
# 该类实现了Bart 模型中的可学习位置编码，并且针对 padding token 做了特殊偏移处理，支持训练和增量推理时的正确位置索引计算。
class BartLearnedPositionalEmbedding(nn.Embedding):
    def __init__(self, num_embeddings: int, embedding_dim: int):
        # Bart 特殊设计，若存在 padding_idx，则将所有位置编号 +2，且扩展嵌入表大小，防止冲突
        self.offset = 2
        super().__init__(num_embeddings + self.offset, embedding_dim)
    # input_ids: [batch_size, seq_len]，输入token的ID序列（不直接用作嵌入索引，仅用来推断形状）
    # past_key_values_length: int，解码器中缓存的过去序列长度，默认0，用于支持增量推理时的位置调整
    def forward(self, input_ids: torch.Tensor, past_key_values_length: int = 0):
        bsz, seq_len = input_ids.shape[:2] # batch_size 和序列长度
        # 生成位置索引张量，范围连续支持缓存长度
        positions = torch.arange(
            past_key_values_length, past_key_values_length + seq_len, dtype=torch.long, device=self.weight.device
        ).expand(bsz, -1)
        # 返回对应位置的可学习嵌入，索引加上 offset 避免和 padding 冲突
        return super().forward(positions + self.offset)

In [17]:
torch.arange(0,8, dtype=torch.long).expand(2, -1)

tensor([[0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7]])

In [18]:
torch.arange(0,8, dtype=torch.long).expand(2, -1)+2

tensor([[2, 3, 4, 5, 6, 7, 8, 9],
        [2, 3, 4, 5, 6, 7, 8, 9]])

In [ ]:
# Bart中加2的原因主要是为了避免位置索引与特殊token的ID冲突，具体原因如下：
# 特殊token预留
# Bart的词表中，padding_token_id通常是0，decoder_start_token_id等特殊token也占用较低的ID（比如1）。
# 为了不让位置编码的索引与这些特殊token的ID混淆，位置编码索引整体往后偏移2。
# 这是Bart实现中的一个“小技巧”，保证padding token的embedding不受位置编码影响，同时简化模型处理逻辑。
# “+2” 是为了保证位置索引与 padding 和特殊token ID 不冲突，避免混淆，保持编码唯一性。

In [19]:
# 继承自 nn.Embedding，重写了 forward 函数，在获取词向量后乘以一个缩放因子 embed_scale。
# 这个缩放操作通常用于平衡词嵌入的数值范围，有助于模型训练稳定性。
class BartScaledWordEmbedding(nn.Embedding):
    def __init__(self, num_embeddings: int, embedding_dim: int, padding_idx: int, embed_scale: Optional[float] = 1.0):
        super().__init__(num_embeddings, embedding_dim, padding_idx)  # 调用父类初始化，设置词表大小、维度和padding索引
        self.embed_scale = embed_scale  # 保存缩放因子，默认1.0，即不缩放

    def forward(self, input_ids: torch.Tensor):
        # 调用父类forward获取对应input_ids的词嵌入
        # 之后乘以缩放因子 embed_scale，实现嵌入向量的缩放
        return super().forward(input_ids) * self.embed_scale

In [20]:
aa=torch.randn((2*3,5,6))

In [21]:
aa.shape

torch.Size([6, 5, 6])

In [22]:
bb=aa.view(2,3,5,6)

In [23]:
bb.shape

torch.Size([2, 3, 5, 6])

In [24]:
aa.shape

torch.Size([6, 5, 6])

In [25]:
# 来自“Attention Is All You Need”论文的多头注意力
class BartAttention(nn.Module):
    def __init__(
        self,
        embed_dim: int,
        num_heads: int,
        dropout: float = 0.0,
        is_decoder: bool = False,
        bias: bool = True,
        is_causal: bool = False,
        config: Optional[BartConfig] = None,
    ):
        super().__init__()
        self.embed_dim = embed_dim  # 嵌入维度
        self.num_heads = num_heads # 注意力头数量
        self.dropout = dropout # dropout 概率
        self.head_dim = embed_dim // num_heads # 每个头的维度
        self.config = config  # 配置参数
        # 检查 embed_dim 是否能被 num_heads 整除
        if (self.head_dim * num_heads) != self.embed_dim:
            raise ValueError(
                f"embed_dim must be divisible by num_heads (got `embed_dim`: {self.embed_dim}"
                f" and `num_heads`: {num_heads})."
            )
        self.scaling = self.head_dim**-0.5  # 缩放因子，防止点积值过大
        self.is_decoder = is_decoder # 是否解码器模式
        self.is_causal = is_causal # 是否因果遮蔽（单向）
         # 定义线性层，分别用于生成 key, value, query 和最终输出
        self.k_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.v_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.q_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
    # 重塑张量形状为 (batch_size, num_heads, seq_len, head_dim)，方便多头并行计算
    def _shape(self, tensor: torch.Tensor, seq_len: int, bsz: int):
        return tensor.view(bsz, seq_len, self.num_heads, self.head_dim).transpose(1, 2).contiguous()

    def forward(
        self,
        hidden_states: torch.Tensor,
        key_value_states: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        # 是否是交叉注意力（decoder中用encoder的key/value）key_value_states应该是编码器的输出
        is_cross_attention = key_value_states is not None
        # batch_size, 目标序列长度
        bsz, tgt_len, _ = hidden_states.size()
        # 计算 query 并乘缩放因子
        query_states = self.q_proj(hidden_states) * self.scaling
         # 根据不同情况计算 key 和 value：
        # 1. 交叉注意力且已有缓存，复用缓存key/value
        if (
            is_cross_attention
            and past_key_value is not None
            and past_key_value[0].shape[2] == key_value_states.shape[1]
        ):
            # 重用k，v，cross_attentions
            key_states = past_key_value[0]
            value_states = past_key_value[1]
        elif is_cross_attention: # 2. 交叉注意力，无缓存，计算 key/value
            # cross_attentions
            key_states = self._shape(self.k_proj(key_value_states), -1, bsz)
            value_states = self._shape(self.v_proj(key_value_states), -1, bsz)
        # 3. 非交叉注意力，且有缓存，复用并拼接缓存key/value（用于decoder的自注意力缓存）
        elif past_key_value is not None:
            # 会先计算当前传入的k,v之后和缓存的k,v在序列维度合并
            key_states = self._shape(self.k_proj(hidden_states), -1, bsz)
            value_states = self._shape(self.v_proj(hidden_states), -1, bsz)
            key_states = torch.cat([past_key_value[0], key_states], dim=2)
            value_states = torch.cat([past_key_value[1], value_states], dim=2)
        else: # 4. 普通自注意力，无缓存，直接计算 key/value
            # self_attention
            key_states = self._shape(self.k_proj(hidden_states), -1, bsz)
            value_states = self._shape(self.v_proj(hidden_states), -1, bsz)
         # 解码器情况下，保存当前 key/value 用于下一步缓存
        # 解码器情况有两种,一种是交叉注意力,一种是有自注意力
        if self.is_decoder: 
            # 如果是交叉注意力，则保存一个包含所有交叉注意力的键（key）和值（value）状态的元组（Tuple(torch.Tensor, torch.Tensor)）。
            # 后续调用交叉注意力层时，可以重用所有已保存的交叉注意力键/值状态（对应第一个 if 分支）。
            # 如果是单向自注意力（解码器），则保存所有之前解码器的键/值状态的元组。后续调用单向自注意力时，可以将之前保存的键/值状态与当前投影的键/值状态拼接（对应第三个 elif 分支）。
            # 如果是编码器的双向自注意力，past_key_value 始终为 None。
            past_key_value = (key_states, value_states)
        # 将 query/key/value reshape成 (batch_size * num_heads, seq_len, head_dim) 方便并行计算
        proj_shape = (bsz * self.num_heads, -1, self.head_dim)
        query_states = self._shape(query_states, tgt_len, bsz).view(*proj_shape)
        key_states = key_states.reshape(*proj_shape)
        value_states = value_states.reshape(*proj_shape)
        src_len = key_states.size(1)  # 源序列长度
        # torch.bmm 是批量矩阵乘法（batch matrix multiplication）。
        # query_states 和 key_states.transpose(1, 2) 都是三维张量，形状一般是 (batch_size * num_heads, seq_len, 
        # head_dim) 和 (batch_size * num_heads, head_dim, seq_len)，bmm 会对每个 batch（这里包含多头的合并）做矩
        # 阵乘法，得到形状 (batch_size * num_heads, seq_len, seq_len) 的注意力权重矩阵。
        # torch.matmul：支持广播，可以对高维张量按最后两维做矩阵乘法。
        attn_weights = torch.bmm(query_states, key_states.transpose(1, 2))  # 计算注意力权重 Q*K^T
        # 检查权重形状是否正确
        if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
            raise ValueError(
                f"Attention weights should be of size {(bsz * self.num_heads, tgt_len, src_len)}, but is"
                f" {attn_weights.size()}"
            )
        # 如果有注意力mask，加上mask（形状为 (batch, 1, tgt_len, src_len)）
        if attention_mask is not None:
            if attention_mask.size() != (bsz, 1, tgt_len, src_len):
                raise ValueError(
                    f"Attention mask should be of size {(bsz, 1, tgt_len, src_len)}, but is {attention_mask.size()}"
                )
            # 加上mask
            attn_weights = attn_weights.view(bsz, self.num_heads, tgt_len, src_len) + attention_mask
            attn_weights = attn_weights.view(bsz * self.num_heads, tgt_len, src_len)
        # 表示query中每个token对key中token的关注分数
        attn_weights = nn.functional.softmax(attn_weights, dim=-1) # 归一化权重为概率分布
        # 如果有头掩码，乘以对应的掩码值，屏蔽某些头 先对权重矩阵归一化,之后乘以头掩码
        if layer_head_mask is not None:
            if layer_head_mask.size() != (self.num_heads,):
                raise ValueError(
                    f"Head mask for a single layer should be of size {(self.num_heads,)}, but is"
                    f" {layer_head_mask.size()}"
                )
            attn_weights = layer_head_mask.view(1, -1, 1, 1) * attn_weights.view(bsz, self.num_heads, tgt_len, src_len)
            attn_weights = attn_weights.view(bsz * self.num_heads, tgt_len, src_len)
        # 是否返回注意力权重  attn_weights_reshaped用于输出权重矩阵,attn_weights用于之后的计算
        if output_attentions:
            # 这句 不会改变原来的 attn_weights 对象，而是：返回一个新的张量对象（新变量 attn_weights_reshaped 指向它）。
            attn_weights_reshaped = attn_weights.view(bsz, self.num_heads, tgt_len, src_len)
            # 这个没必要,因为attn_weights指向的那个对象压根没变化
            # attn_weights = attn_weights_reshaped.view(bsz * self.num_heads, tgt_len, src_len)
        else: # 这个是output_attentions=False的情况
            attn_weights_reshaped = None
        # Dropout 正则化
        attn_probs = nn.functional.dropout(attn_weights, p=self.dropout, training=self.training)
        # 计算加权和，得到注意力输出
        attn_output = torch.bmm(attn_probs, value_states)
        # 检查输出形状
        if attn_output.size() != (bsz * self.num_heads, tgt_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(bsz * self.num_heads, tgt_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )
        # 恢复成 (batch, num_heads, tgt_len, head_dim)
        attn_output = attn_output.view(bsz, self.num_heads, tgt_len, self.head_dim)
        attn_output = attn_output.transpose(1, 2) # 转换成 (batch, tgt_len, num_heads, head_dim)

        # 合并多头维度，恢复到 embed_dim
        attn_output = attn_output.reshape(bsz, tgt_len, self.embed_dim)
        # 通过输出线性层，得到最终输出
        attn_output = self.out_proj(attn_output)
        # 返回：输出张量，注意力权重（可选），缓存的key/value（用于解码器）
        return attn_output, attn_weights_reshaped, past_key_value

In [ ]:
# 因为 torch.bmm（用于乘 value_states）只能接受形状为 (bsz * num_heads, tgt_len, src_len)，所以必须在乘法前变成
# 这种扁平化的 shape。中间这一段只是为了 输出需要 reshape 一次保存副本，但仍需要保持原 shape 做后续计算。
# 所以虽然看似重复，实则是为了分离：
# 计算用的 attn_weights
# 输出用的 attn_weights_reshaped

In [ ]:
@lru_cache()
def is_flash_attn_greater_or_equal_2_10():
    if not _is_package_available("flash_attn"):
        return False
    return version.parse(importlib.metadata.version("flash_attn")) >= version.parse("2.1.0")

In [26]:
from transformers.utils import is_flash_attn_greater_or_equal_2_10

In [27]:
is_flash_attn_greater_or_equal_2_10()

True

In [ ]:
def flash_attn_supports_top_left_mask():
    if is_flash_attn_2_available():
        # top-left mask is used in package `flash-attn` with version lower than 2.1.0
        return not is_flash_attn_greater_or_equal_2_10()

    if is_torch_npu_available():
        # down-right mask is used on Ascend NPU by default, set env `NPU_FA2_SPARSE_MODE=2` to activate top-left mask.
        from .integrations.npu_flash_attention import is_npu_fa2_top_left_aligned_causal_mask

        return is_npu_fa2_top_left_aligned_causal_mask()

    return False

In [28]:
flash_attn_supports_top_left_mask()

False

In [29]:
bb.shape

torch.Size([2, 3, 5, 6])

In [30]:
cc2 = bb.transpose(1, 2)

In [31]:
print(bb.shape,cc2.shape)

torch.Size([2, 3, 5, 6]) torch.Size([2, 5, 3, 6])


In [32]:
torch.get_autocast_gpu_dtype()

/tmp/ipykernel_35/3134095016.py:1: DeprecationWarning: torch.get_autocast_gpu_dtype() is deprecated. Please use torch.get_autocast_dtype('cuda') instead. (Triggered internally at /pytorch/torch/csrc/autograd/init.cpp:787.)
  torch.get_autocast_gpu_dtype()


torch.float16

In [33]:
torch.is_autocast_enabled() # 是否支持混合精度

False

In [34]:
# 用于 BART 的 Flash Attention 模块。
# - 继承自 BartAttention，因此保留了权重结构（如 q_proj/k_proj/v_proj/out_proj）不变；
# - 仅重写 forward 方法以调用 FlashAttention 的公开 API；
# - 同时处理 FlashAttention 对不同版本（如 2.0 与 2.1）掩码方式差异的问题；
# - 支持自注意力与交叉注意力，同时能复用 past_key_value 以支持增量推理。
class BartFlashAttention2(BartAttention):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # 处理 FlashAttention <2.1 所使用的“左上角对齐”的 causal mask（早期实现方式）
        # 从 FlashAttention 2.1 开始，掩码行为改为默认支持“右下角对齐”，更符合常规因果 Mask。
        # 该标志用于控制 forward 中是否需要兼容老版本行为。
        self._flash_attn_uses_top_left_mask = flash_attn_supports_top_left_mask()
    # 重写了父类方法 将投影后的 hidden_states reshape 成 (bsz, seq_len, num_heads, head_dim)
    def _reshape(self, tensor: torch.Tensor, seq_len: int, bsz: int):
        return tensor.view(bsz, seq_len, self.num_heads, self.head_dim)

    def forward(
        self,
        hidden_states: torch.Tensor,
        key_value_states: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
         # flash-attn 不支持输出注意力权重（因为它不显式计算注意力矩阵），因此禁止该模式
        if output_attentions:
            raise ValueError("BartFlashAttention2 attention does not support output_attentions")

        # 是否是 cross-attention key_value_states应该是编码器的输出
        is_cross_attention = key_value_states is not None
        bsz, q_len, _ = hidden_states.size() # 目标序列批次和序列长度
        # 计算 query 投影并 reshape 成 (bsz, q_len, num_heads, head_dim)
        query_states = self._reshape(self.q_proj(hidden_states), -1, bsz)
        # 根据是否 cross-attention 与是否提供 past_key_value 分支处理 key/value 的获取
        # 如果是交叉注意力并且past_key_value存在
        if (
            is_cross_attention
            and past_key_value is not None
            and past_key_value[0].shape[2] == key_value_states.shape[1]
        ):
            # 直接用past_key_value,换轴为(b,s,h,hd)
            key_states = past_key_value[0].transpose(1, 2)
            value_states = past_key_value[1].transpose(1, 2)
        elif is_cross_attention: # 是交叉注意力,但是没缓存
            # 根据编码器的输出重新计算key,vlue
            key_states = self._reshape(self.k_proj(key_value_states), -1, bsz)
            value_states = self._reshape(self.v_proj(key_value_states), -1, bsz)
        elif past_key_value is not None: # 是自注意力并且存在缓存
            # 先计算当前传入的单个token的key和value
            key_states = self._reshape(self.k_proj(hidden_states), -1, bsz)
            value_states = self._reshape(self.v_proj(hidden_states), -1, bsz)
            # 之后在序列维度合并 
            key_states = torch.cat([past_key_value[0].transpose(1, 2), key_states], dim=1)
            value_states = torch.cat([past_key_value[1].transpose(1, 2), value_states], dim=1)
        else: # 是自注意力,并且没有缓存
            # 根据hidden_states重新计算
            key_states = self._reshape(self.k_proj(hidden_states), -1, bsz)
            value_states = self._reshape(self.v_proj(hidden_states), -1, bsz)
        # 如果是 decoder，保存 k/v 以供后续时间步或交叉注意力使用
        if self.is_decoder:
            # 如果是交叉注意力，则保存一个包含所有交叉注意力的键（key）和值（value）状态的元组（Tuple(torch.Tensor, torch.Tensor)）。
            # 后续调用交叉注意力层时，可以重用所有已保存的交叉注意力键/值状态（对应第一个 if 分支）。
            # 如果是单向自注意力（解码器），则保存所有之前解码器的键/值状态的元组。后续调用单向自注意力时，可以将之前保存的键/值状态
            # 与当前投影的键/值状态拼接（对应第三个 elif 分支）。
            # 如果是编码器的双向自注意力，past_key_value 始终为 None。
            # key_states.transpose(1, 2) 本身不修改原 tensor 的内存布局（它是 创建一个新的 view）
            # past_key_value 是一个新变量，指向新的 transpose 后的 tensor 对象
            # 原始的 key_states 并没有被修改，因为它本身没有被就地 (in-place) 改动
            past_key_value = (key_states.transpose(1, 2), value_states.transpose(1, 2))
        # key_states.shape[-2] 是 num_heads，不应该作为 kv_seq_len
        # 应该写成 key_states.shape[1] 才对（因为 shape 是 (bsz, seq_len, num_heads, head_dim)）
        # kv_seq_len = key_states.shape[-2] 
        # 这段代码用意可能是想获得完整的 kv 序列长度，
        # 但由于代码中 key_states 已经包含了缓存的拼接，直接加上缓存长度会造成重复累加，
        # 因此，这两行代码要么需要重写（改成直接用 kv_seq_len = key_states.shape[-2]），
        # 要么就是冗余、无意义的，可以去掉。
        # kv_seq_len = key_states.shape[1]
        # if past_key_value is not None:
        #     kv_seq_len += past_key_value[0].shape[-2]

        # 在 PEFT（参数高效微调）中，通常会将 LayerNorm 层转换为 float32 类型，以提高训练的稳定性。
        # 因此，输入的 hidden_states 会被悄悄地转换为 float32 类型。
        # 所以我们需要将它们重新转换回正确的 dtype，以确保一切按预期工作。
        # 但这可能会降低训练和推理的速度，因此不建议将 LayerNorm 转换为 float32。
        # （LlamaRMSNorm 处理得当，不会有这个问题）
        # 检查输入数据类型，如果是 float32，但推理处于 autocast 模式或权重是 float16，则回退 cast
        input_dtype = query_states.dtype
        if input_dtype == torch.float32:
            # torch.is_autocast_enabled() 用来判断当前上下文环境中 自动混合精度（autocast） 是否处于启用状态。
            # 自动混合精度是 PyTorch 用于加速训练和推理的一种机制，它会自动在适当的操作中使用半精度（float16）计算，从而提升效率并降低显存占用。
            if torch.is_autocast_enabled():
                target_dtype = torch.get_autocast_gpu_dtype() # 设置目标类型
            # 处理模型量化的情况
            elif hasattr(self.config, "_pre_quantization_dtype"):
                target_dtype = self.config._pre_quantization_dtype
            else:
                target_dtype = self.q_proj.weight.dtype

            logger.warning_once(
                f"The input hidden states seems to be silently casted in float32, this might be related to"
                f" the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in"
                f" {target_dtype}."
            )
            # 转换q,k,v的类型为target_dtype
            query_states = query_states.to(target_dtype)
            key_states = key_states.to(target_dtype)
            value_states = value_states.to(target_dtype)
        # 调用_flash_attention_forward函数计算注意力输出
        attn_output = _flash_attention_forward(
            query_states,  # 查询张量 (q)
            key_states,  # 键张量 (k)
            value_states,  # 值张量 (v)
            attention_mask,  # 注意力掩码，屏蔽不应关注的部分
            q_len,  # 查询序列长度
            dropout=self.dropout if self.training else 0.0, # 训练时应用dropout，推理时不使用
            is_causal=self.is_causal,   # 是否使用因果掩码（未来信息屏蔽）
            use_top_left_mask=self._flash_attn_uses_top_left_mask, # 是否启用特殊的Top-Left掩码
        )
        # 将注意力输出张量reshape为(batch_size, query_length, 隐藏层维度)
        attn_output = attn_output.reshape(bsz, q_len, -1)
        attn_output = self.out_proj(attn_output) # 经过输出投影层，映射回隐藏层维度
        # 如果不需要返回注意力权重，则置为None，节省内存
        if not output_attentions:
            attn_weights = None
        # 返回注意力输出，注意力权重（可能为None），以及缓存的past_key_value（用于加速推理）
        return attn_output, attn_weights, past_key_value

In [35]:
def _flash_attention_forward(
    query_states: torch.Tensor,
    key_states: torch.Tensor,
    value_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    query_length: int,
    is_causal: bool,
    dropout: float = 0.0,
    position_ids: Optional[torch.Tensor] = None,
    softmax_scale: Optional[float] = None,
    sliding_window: Optional[int] = None,
    use_top_left_mask: bool = False,
    softcap: Optional[float] = None,
    deterministic: Optional[bool] = None,
    cu_seq_lens_q: Optional[torch.LongTensor] = None,
    cu_seq_lens_k: Optional[torch.LongTensor] = None,
    max_length_q: Optional[int] = None,
    max_length_k: Optional[int] = None,
    target_dtype: Optional[torch.dtype] = None,
    **kwargs,
):
    """
    Calls the forward method of Flash Attention - if the input hidden states contain at least one padding token
    first unpad the input, then computes the attention scores and pad the final attention scores.

    Args:
        query_states (`torch.Tensor`):
            Input query states to be passed to Flash Attention API
        key_states (`torch.Tensor`):
            Input key states to be passed to Flash Attention API
        value_states (`torch.Tensor`):
            Input value states to be passed to Flash Attention API
        attention_mask (`torch.Tensor`, *optional*):
            The padding mask - corresponds to a tensor of size `(batch_size, seq_len)` where 0 stands for the
            position of padding tokens and 1 for the position of non-padding tokens.
        dropout (`float`):
            Attention dropout
        softmax_scale (`float`, *optional*):
            The scaling of QK^T before applying softmax. Default to 1 / sqrt(head_dim)
        use_top_left_mask (`bool`, defaults to `False`):
            flash_attn<2.1 generates top-left aligned causal mask, while what is needed here is bottom-right alignment, that was made default for flash_attn>=2.1. This attribute is used to handle this difference.
        softcap (`float`, *optional*):
            Softcap for the attention logits, used e.g. in gemma2.
        deterministic (`bool`, *optional*):
            Determines if the deterministic option introduced in flash_attn>=2.4.1 is enabled.
    """
    if not use_top_left_mask:
        causal = is_causal
    else:
        # TODO: Remove the `query_length != 1` check once Flash Attention for RoCm is bumped to 2.1.
        causal = is_causal and query_length != 1

    # Assuming 4D tensors, key_states.shape[1] is the key/value sequence length (source length).
    use_sliding_windows = (
        _flash_supports_window_size and sliding_window is not None and key_states.shape[1] > sliding_window
    )
    flash_kwargs = {"window_size": (sliding_window, sliding_window)} if use_sliding_windows else {}

    if flash_241:
        if deterministic is None:
            deterministic = deterministic_g
        flash_kwargs["deterministic"] = deterministic

    if softcap is not None:
        flash_kwargs["softcap"] = softcap

    # PEFT possibly silently casts tensors to fp32, this potentially reconverts to correct dtype or is a no op
    query_states, key_states, value_states = fa_peft_integration_check(
        query_states, key_states, value_states, target_dtype
    )

    # Contains at least one padding token in the sequence
    if attention_mask is not None:
        batch_size = query_states.shape[0]
        query_states, key_states, value_states, indices_q, cu_seq_lens, max_seq_lens = _upad_input(
            query_states, key_states, value_states, attention_mask, query_length
        )
        cu_seqlens_q, cu_seqlens_k = cu_seq_lens
        max_seqlen_in_batch_q, max_seqlen_in_batch_k = max_seq_lens

        attn_output_unpad = flash_attn_varlen_func(
            query_states,
            key_states,
            value_states,
            cu_seqlens_q=cu_seqlens_q,
            cu_seqlens_k=cu_seqlens_k,
            max_seqlen_q=max_seqlen_in_batch_q,
            max_seqlen_k=max_seqlen_in_batch_k,
            dropout_p=dropout,
            softmax_scale=softmax_scale,
            causal=causal,
            **flash_kwargs,
        )
        attn_output = pad_input(attn_output_unpad, indices_q, batch_size, query_length)

    # If position_ids is provided and check all examples do not contain only 1 sequence, If tensor in increasing
    # then we probably have one sequence, otherwise it is packed. Additionally check we are in pre-fill/training stage.
    # Use `flash_attn_varlen_func` to prevent cross-example attention and also allow padding free approach
    elif position_ids is not None and (
        max_length_q is not None or (query_length != 1 and not (torch.diff(position_ids, dim=-1) >= 0).all())
    ):
        batch_size = query_states.size(0)

        if cu_seq_lens_q is None or cu_seq_lens_k is None:
            query_states, key_states, value_states, indices_q, cu_seq_lens, max_seq_lens = (
                prepare_fa2_from_position_ids(query_states, key_states, value_states, position_ids)
            )

            cu_seq_lens_q, cu_seq_lens_k = cu_seq_lens
            max_length_q, max_length_k = max_seq_lens

        else:
            query_states = query_states.reshape(-1, query_states.size(-2), query_states.size(-1))
            key_states = key_states.reshape(-1, key_states.size(-2), key_states.size(-1))
            value_states = value_states.reshape(-1, value_states.size(-2), value_states.size(-1))

        attn_output = flash_attn_varlen_func(
            query_states,
            key_states,
            value_states,
            cu_seqlens_q=cu_seq_lens_q,
            cu_seqlens_k=cu_seq_lens_k,
            max_seqlen_q=max_length_q,
            max_seqlen_k=max_length_k,
            dropout_p=dropout,
            softmax_scale=softmax_scale,
            causal=causal,
            **flash_kwargs,
        )

        attn_output = attn_output.view(batch_size, -1, attn_output.size(-2), attn_output.size(-1))

    else:
        attn_output = flash_attn_func(
            query_states, key_states, value_states, dropout, softmax_scale=softmax_scale, causal=causal, **flash_kwargs
        )

    return attn_output

In [36]:
class BartSdpaAttention(BartAttention):
    def forward(
        self,
        hidden_states: torch.Tensor,
        key_value_states: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        # 输入维度: [batch_size, seq_len, hidden_dim]
        # 若启用输出注意力或head mask，因SDPA不支持这些功能，回退到手动实现
        if output_attentions or layer_head_mask is not None:
            logger.warning_once(
                "BartModel is using BartSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention"
                ' implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.'
            )
            return super().forward( # 回退到BartAttention的实现
                hidden_states,
                key_value_states=key_value_states,
                past_key_value=past_key_value,
                attention_mask=attention_mask,
                layer_head_mask=layer_head_mask,
                output_attentions=output_attentions,
            )

        # 判断是否是交叉注意力（key_value_states是encoder输出）
        is_cross_attention = key_value_states is not None

        bsz, tgt_len, _ = hidden_states.size() # 获取目标的批次和序列长度

        # 计算query投影
        query_states = self.q_proj(hidden_states)
        # 交叉注意力,并且有缓存k,v
        if (
            is_cross_attention
            and past_key_value is not None
            and past_key_value[0].shape[2] == key_value_states.shape[1]
        ):
            # 重用缓存中的key/value
            key_states = past_key_value[0]
            value_states = past_key_value[1]
        elif is_cross_attention:
            # 交叉注意力,但是没有past_k，v,这时投影变形成(b,h,s,hd)形式
            key_states = self._shape(self.k_proj(key_value_states), -1, bsz)
            value_states = self._shape(self.v_proj(key_value_states), -1, bsz)
        elif past_key_value is not None:
            # decoder自注意力,有缓存
            # 先计算当前输入中的目标序列的k,v表示（b,h,s,hd）
            key_states = self._shape(self.k_proj(hidden_states), -1, bsz)
            value_states = self._shape(self.v_proj(hidden_states), -1, bsz)
            # 之后再序列轴合并
            key_states = torch.cat([past_key_value[0], key_states], dim=2)
            value_states = torch.cat([past_key_value[1], value_states], dim=2)
        else:
            # 如果不是交叉注意力,并且没有缓存 (b,h,s,hd)
            key_states = self._shape(self.k_proj(hidden_states), -1, bsz)
            value_states = self._shape(self.v_proj(hidden_states), -1, bsz)
        # 如果是decoder 则缓存k,v 否则如果是单编码器,past_key_value =None
        if self.is_decoder:
            # 保存当前key/value以供下一步使用（缓存）
            past_key_value = (key_states, value_states)
        # 将query reshape 成 [batch_size, num_heads, tgt_len, head_dim]
        query_states = self._shape(query_states, tgt_len, bsz)

        # 我们通过这个 is_causal 条件语句来分发调用 SDPA（Scaled Dot Product Attention）中的 Flash Attention 或 Efficient 
        #  内核，而不是在 SDPA 内部使用内联条件判断，这是为了兼容 torch.compile 的动态形状和完整图优化。内联条件会阻碍动态形状编译。
        #  tgt_len > 1 是必要的，这是为了与 AttentionMaskConverter.to_causal_4d 的行为保持一致——当 tgt_len == 1 时，该方
        #  法不会创建因果掩码。
        # 如果is_causal标记是True,并且没有传入填充掩码,并且当前输入序列长度大于1(就是没有缓存k,v)
        is_causal = True if self.is_causal and attention_mask is None and tgt_len > 1 else False

        # 使用 PyTorch 原生 SDPA（支持 FlashAttention / Memory Efficient Attention） 缩放点积注意力机制
        attn_output = torch.nn.functional.scaled_dot_product_attention(
            query_states,
            key_states,
            value_states,
            attn_mask=attention_mask,
            dropout_p=self.dropout if self.training else 0.0,
            is_causal=is_causal,
        )
         # 校验输出形状
        if attn_output.size() != (bsz, self.num_heads, tgt_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(bsz, self.num_heads, tgt_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )
        # 变换维度：[batch, heads, seq_len, head_dim] -> [batch, seq_len, heads, head_dim]
        attn_output = attn_output.transpose(1, 2)
         # 合并多头结果为 [batch, seq_len, embed_dim]
        attn_output = attn_output.reshape(bsz, tgt_len, self.embed_dim)
         # 输出线性映射
        attn_output = self.out_proj(attn_output)
         # 返回注意力输出、注意力权重（此处为None）、缓存的key/value
        return attn_output, None, past_key_value

In [ ]:
# 我们通过这个 is_causal 条件语句来分发调用 SDPA（Scaled Dot Product Attention）中的 Flash Attention 或 Efficient 
# 内核，而不是在 SDPA 内部使用内联条件判断，这是为了兼容 torch.compile 的动态形状和完整图优化。内联条件会阻碍动态形状编译。
# tgt_len > 1 是必要的，这是为了与 AttentionMaskConverter.to_causal_4d 的行为保持一致——当 tgt_len == 1 时，该方法不会创建因果掩码。

In [37]:
# BART各种注意力类
BART_ATTENTION_CLASSES = {
    "eager": BartAttention,
    "sdpa": BartSdpaAttention,
    "flash_attention_2": BartFlashAttention2,
}

In [38]:
from transformers import BartConfig

In [39]:
config=BartConfig()

In [40]:
config._attn_implementation

'eager'

In [41]:
class BartEncoderLayer(nn.Module):
    def __init__(self, config: BartConfig):
        super().__init__()
        self.embed_dim = config.d_model # 嵌入维度（模型宽度）
        # 初始化自注意力层（支持 Flash Attention / SDPA 等多种实现）
        self.self_attn = BART_ATTENTION_CLASSES[config._attn_implementation](
            embed_dim=self.embed_dim,
            num_heads=config.encoder_attention_heads,
            dropout=config.attention_dropout,
            config=config,
        )
        self.self_attn_layer_norm = nn.LayerNorm(self.embed_dim) # 自注意力后的 LayerNorm
        self.dropout = config.dropout  # 残差连接后的 dropout 概率
        self.activation_fn = ACT2FN[config.activation_function] # 非线性激活函数（如 GELU、ReLU）
        self.activation_dropout = config.activation_dropout # FFN 层激活后的 dropout 概率
        self.fc1 = nn.Linear(self.embed_dim, config.encoder_ffn_dim) # 前馈神经网络第1层：升维
        self.fc2 = nn.Linear(config.encoder_ffn_dim, self.embed_dim)  # 前馈神经网络第2层：降维
        self.final_layer_norm = nn.LayerNorm(self.embed_dim) # FFN 后的 LayerNorm

    def forward(
        self,
        hidden_states: torch.FloatTensor,  # 输入隐藏状态：(batch, seq_len, embed_dim)
        attention_mask: torch.FloatTensor, # 注意力掩码：(batch, 1, tgt_len, src_len)，用于屏蔽 padding
        layer_head_mask: torch.FloatTensor,  # 指定当前层中哪些注意力头被禁用
        output_attentions: Optional[bool] = False,  # 是否返回注意力矩阵
    ) -> Tuple[torch.FloatTensor, Optional[torch.FloatTensor]]:
        # ----------- 自注意力子层（含残差和 LayerNorm） ----------- #
        residual = hidden_states  # 残差连接的输入
        hidden_states, attn_weights, _ = self.self_attn(  # 执行自注意力
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            layer_head_mask=layer_head_mask,
            output_attentions=output_attentions,
        )
        # dropout
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)
        hidden_states = residual + hidden_states # 残差连接
        hidden_states = self.self_attn_layer_norm(hidden_states) # LayerNorm
         # ----------- 前馈网络子层（含残差和 LayerNorm） ----------- #
        residual = hidden_states  # 残差连接的输入
        hidden_states = self.activation_fn(self.fc1(hidden_states)) # 第1层线性+激活
        hidden_states = nn.functional.dropout(hidden_states, p=self.activation_dropout, training=self.training) # dropout
        hidden_states = self.fc2(hidden_states) # 第2层线性
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training) # dropout
        hidden_states = residual + hidden_states # 残差连接
        hidden_states = self.final_layer_norm(hidden_states) # LayerNorm
        # ----------- 数值稳定性防护（float16时） ----------- #
        if hidden_states.dtype == torch.float16 and (
            torch.isinf(hidden_states).any() or torch.isnan(hidden_states).any()
        ):
            # 避免溢出：截断数值以防出现无穷大/NaN
            # clamp(input, min=None, max=None, *, out=None) -> Tensor
            # 将 input 中的所有元素限制在区间 [min, max] 之间
            # torch.clamp 和 numpy.clip 在功能上是等价的
            clamp_value = torch.finfo(hidden_states.dtype).max - 1000
            hidden_states = torch.clamp(hidden_states, min=-clamp_value, max=clamp_value)

        outputs = (hidden_states,)   # 主输出是编码后的 hidden states

        if output_attentions:
            outputs += (attn_weights,)  # 如果需要，返回注意力权重

        return outputs # 返回：编码后的 hidden states（可选 attention 权重）

In [42]:
import numpy as np

In [43]:
torch.isinf(torch.from_numpy(np.array([3,6,1000000]))).any()

tensor(False)

In [44]:
torch.isnan(torch.from_numpy(np.array([3,6,1000000]))).any()

tensor(False)

In [45]:
torch.from_numpy(np.array([3.,6.,1000000.])).dtype

torch.float64

In [46]:
torch.finfo(torch.float64).max

1.7976931348623157e+308

In [49]:
# 封装就是层层嵌套调用,这里会用到之前定义的作为子模块
class BartDecoderLayer(nn.Module):
    def __init__(self, config: BartConfig):
        super().__init__()
        self.embed_dim = config.d_model # 隐藏层维度（即embedding维度）
        # Decoder中的自注意力机制，支持因果遮盖（即只能看到过去）
        self.self_attn = BART_ATTENTION_CLASSES[config._attn_implementation](
            embed_dim=self.embed_dim,
            num_heads=config.decoder_attention_heads,
            dropout=config.attention_dropout,
            is_decoder=True, # 默认是False,这里设定是解码器
            is_causal=True, # 设定用因果掩码
            config=config,
        )
        self.dropout = config.dropout # dropout比率
        self.activation_fn = ACT2FN[config.activation_function] # 非线性激活函数
        self.activation_dropout = config.activation_dropout # FFN层的dropout
        # 自注意力之后的LayerNorm
        self.self_attn_layer_norm = nn.LayerNorm(self.embed_dim)
        # 编码器-解码器交叉注意力
        self.encoder_attn = BART_ATTENTION_CLASSES[config._attn_implementation](
            self.embed_dim,
            config.decoder_attention_heads,
            dropout=config.attention_dropout,
            is_decoder=True, # 这里是解码器的交叉注意力
            config=config,
        )
        self.encoder_attn_layer_norm = nn.LayerNorm(self.embed_dim)  # 交叉注意力后的LayerNorm
        self.fc1 = nn.Linear(self.embed_dim, config.decoder_ffn_dim)  # 前馈网络（第一层）：升维
        self.fc2 = nn.Linear(config.decoder_ffn_dim, self.embed_dim) # 前馈网络（第二层）：降维回原始维度
        self.final_layer_norm = nn.LayerNorm(self.embed_dim) # 前馈模块后的最终LayerNorm
    
    # 执行Decoder Layer的前向传播，包含三大模块：
    #     - 自注意力
    #     - 交叉注意力
    #     - 前馈神经网络
    #     支持cache机制（past_key_value）用于加速生成。
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.Tensor] = None,
        encoder_attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        cross_attn_layer_head_mask: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        output_attentions: Optional[bool] = False,
        use_cache: Optional[bool] = True,
    ) -> Tuple[torch.FloatTensor, Optional[Tuple[torch.FloatTensor, torch.FloatTensor]]]:
        
        residual = hidden_states  # 残差连接保存输入

        # ========== 自注意力模块 ==========
        self_attn_past_key_value = past_key_value[:2] if past_key_value is not None else None
        # 将当前 self-attn 缓存添加到 present_key_value 元组的 1,2 位置
        hidden_states, self_attn_weights, present_key_value = self.self_attn(
            hidden_states=hidden_states,
            past_key_value=self_attn_past_key_value,
            attention_mask=attention_mask,
            layer_head_mask=layer_head_mask,
            output_attentions=output_attentions,
        )
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training) # dropout
        hidden_states = residual + hidden_states  # 残差连接
        hidden_states = self.self_attn_layer_norm(hidden_states) # 层标准化

        # ========== 交叉注意力模块（如果有encoder输出） ==========
        cross_attn_present_key_value = None # 交叉注意力缓存的k,v
        cross_attn_weights = None
        if encoder_hidden_states is not None: # 如果有编码器的输出,这时解码器就有交叉注意部分
            residual = hidden_states # 残差部分

            # cross_attn缓存的key/values元组位于present_key_value元组的3,4位置
            cross_attn_past_key_value = past_key_value[-2:] if past_key_value is not None else None
            hidden_states, cross_attn_weights, cross_attn_present_key_value = self.encoder_attn(
                hidden_states=hidden_states,
                key_value_states=encoder_hidden_states,
                attention_mask=encoder_attention_mask,
                layer_head_mask=cross_attn_layer_head_mask,
                past_key_value=cross_attn_past_key_value, # 交叉注意机制缓存的k,v
                output_attentions=output_attentions,
            )
            hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)
            hidden_states = residual + hidden_states # 残差连接
            hidden_states = self.encoder_attn_layer_norm(hidden_states) # 层标准化

            # 将 cross-attn 添加到 present_key_value 元组的 3,4 位置
            present_key_value = present_key_value + cross_attn_present_key_value # 更新缓存

        # ========== 前馈神经网络 ==========
        residual = hidden_states # 残差
        hidden_states = self.activation_fn(self.fc1(hidden_states)) # 非线性变换（升维）
        hidden_states = nn.functional.dropout(hidden_states, p=self.activation_dropout, training=self.training)
        hidden_states = self.fc2(hidden_states)  # 降维
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)
        hidden_states = residual + hidden_states # 残差连接
        hidden_states = self.final_layer_norm(hidden_states) # 层标准化
        # ========== 返回结果 ==========
        outputs = (hidden_states,)
        # 如果设定要输出注意力权重
        if output_attentions:
            outputs += (self_attn_weights, cross_attn_weights)
        # 如果use_cache为True,就加上缓存,返回的是元组形式
        if use_cache:
            outputs += (present_key_value,)

        return outputs

In [48]:
# 用于句子级别分类任务的分类头，常用于文本分类、情感分析等场景
class BartClassificationHead(nn.Module):
    def __init__(
        self,
        input_dim: int,  # 输入特征维度，通常为 encoder/decoder 输出的隐藏层维度
        inner_dim: int, # 分类头中间层的隐藏维度
        num_classes: int,  # 输出类别数
        pooler_dropout: float, # dropout 概率，用于正则化
     ):
        super().__init__()
        self.dense = nn.Linear(input_dim, inner_dim) # 全连接层：将输入映射到中间隐藏维度，提升表达能力
        self.dropout = nn.Dropout(p=pooler_dropout) # Dropout：防止过拟合
        self.out_proj = nn.Linear(inner_dim, num_classes) # 输出层：将隐藏表示映射为分类 logits

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        # 通常输入为 [CLS] token 或池化后的句向量表示 (b,d)
        # 第一次 Dropout：对输入进行正则化
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.dense(hidden_states) # 非线性变换前的线性层
        hidden_states = torch.tanh(hidden_states) # 非线性激活函数 tanh：增强模型表达能力（与 BERT 的 pooler 相似）
        hidden_states = self.dropout(hidden_states) # 第二次 Dropout：进一步正则化
        hidden_states = self.out_proj(hidden_states)  # 输出层，得到分类 logits
        return hidden_states

In [50]:
class BartPreTrainedModel(PreTrainedModel):
    config_class = BartConfig # 指定使用的配置类（即与该模型关联的 config 类）
    base_model_prefix = "model" # 用于命名权重时的前缀，例如 model.encoder.xxx
    supports_gradient_checkpointing = True # 是否支持梯度检查点（用于节省显存）
    # 加载模型时忽略这两个键的未预期警告（通常这些键是非必要的版本标识）
    # 某些模型权重文件中可能包含额外的 key，例如保存时模型代码中存在 encoder.version 和 decoder.version，
    # 但当前加载时的模型类没有这两个字段。
    # 如果不加这个字段，调用 model.load_state_dict() 时就会触发 "Unexpected key(s) in state_dict" 的警告或报错。
    # 设置这个属性后，加载过程会自动跳过这些 key，避免报错。
    _keys_to_ignore_on_load_unexpected = ["encoder.version", "decoder.version"]
    # 指定哪些模块不应该在模型并行或保存时被拆分（用于加速器或分布式训练）
    # ✅ 在 save_pretrained 时，会考虑哪些模块保持整体保存
    # _no_split_modules 指的是静态结构的拆分保护，而不是权重分片。
    # _no_split_modules 控制的是 结构上能不能把某些层拆分为子模块；
    # 和 文件存储方式无关，也和 多 GPU 训练时的权重分布无关。
    _no_split_modules = [r"BartEncoderLayer", r"BartDecoderLayer"]
    # 指定在加载模型时跳过某些键的设备自动迁移（用于缓存 past_key_values）
    # 当你用 from_pretrained() 加载权重到 GPU（例如 model.to("cuda")）时，transformers 库
    # 会遍历每个权重参数，决定是否迁移设备。如果名字中包含 past_key_values，就会 跳过这个步骤，避免无效迁移或报错。
    _skip_keys_device_placement = "past_key_values"
    # 指定是否支持 Flash Attention v2（高效注意力机制，需硬件支持）
    _supports_flash_attn_2 = True
    # 指定是否支持 PyTorch 的 SDPA（Scaled Dot-Product Attention）
    _supports_sdpa = True
    # 模型权重初始化方法，确保不同模块具有合适的初始化方式
    def _init_weights(self, module):
        std = self.config.init_std # 读取配置中指定的初始化标准差
        if isinstance(module, nn.Linear):
            # 线性层权重正态分布初始化，偏置初始化为0
            module.weight.data.normal_(mean=0.0, std=std)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            # 词嵌入初始化，padding_idx 对应位置设为0（不会影响模型输出）
            module.weight.data.normal_(mean=0.0, std=std)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
    # 构造用于测试或模型追踪的伪输入（dummy inputs），常用于：
    #     - 推理导出（如 ONNX/torchscript）
    #     - 测试模型结构是否正常
    @property
    def dummy_inputs(self):
        pad_token = self.config.pad_token_id   # 获取 pad token 的 ID
        # 构造示例 input_ids，其中每行是一条伪输入序列
        input_ids = torch.tensor([[0, 6, 10, 4, 2], [0, 8, 12, 2, pad_token]], device=self.device)
        # attention_mask 为非 pad 位置设置为 True,填充位置为False
        # ne 是 not equal（不等于）的缩写。
        # input_ids.ne(pad_token) 等价于 input_ids != pad_token。
        dummy_inputs = {
            "attention_mask": input_ids.ne(pad_token),
            "input_ids": input_ids,
        }
        return dummy_inputs

In [ ]:
# 为什么要跳过它？ _skip_keys_device_placement = "past_key_values"
# 因为 past_key_values：
# 通常是 缓存中间状态（如 Transformer 解码器中 self-attention 的 KV 缓存）；
# 在预训练模型的权重文件里一般 没有这个字段，或者它是运行时动态生成的；
# 强行对它进行 .to(device) 会导致 出错或无意义操作。
# 当加载模型权重时，跳过将 past_key_values 对应的权重强制移动到目标设备（如 GPU），即：
# 如果权重的键名中包含 "past_key_values"，则不要显式地为它设置 .to(device)。

In [ ]:
# 🔍 区分两个概念：
# 名称	说明	和 _no_split_modules 的关系
# 静态结构拆分（Module Splitting）	指将一个大的子模块（如 BartEncoderLayer）打断为多个子子模块	
# ✅ 这个会参考 _no_split_modules，防止被拆开
# 权重分片（Checkpoint Sharding）	仅仅把模型权重保存到多个 .bin 文件中，如 
# pytorch_model-00001-of-00005.bin	❌ 这个和 _no_split_modules 无关，只影响 I/O 层面的读
# 这里的“权重分片（sharding）”是指 存储时的分片，不是指运行时的设备间分片。
# 🔍 具体说明：
# 类型	说明	是否和 _no_split_modules 有关
# 存储分片（Storage Sharding）	把 pytorch_model.bin 拆成多个文件，比如 pytorch_model-00001-of-00005.bin。常用于大模型存储。	❌ 无关
# 运行时分片（Runtime Sharding）	像 FSDP（Fully Sharded Data Parallel）那样，在多个 GPU 之间分配部分权重，仅在需要时通信	❌ 也无关
# 模块结构拆分（Module Splitting）	分析 nn.Module 的结构，拆分成更小的子块，常用于部署或模型转换	✅ _no_split_modules 就是防止这类结构性拆分

In [ ]:
# 拆分会做什么？
# 以 FSDP 为例，它可能会将如下结构拆分：
# Sequential(
#     LayerNorm,
#     Attention,
#     LayerNorm,
#     MLP
# )
# 为多个部分分配到多个 GPU 上，但 不能拆分一个 BartEncoderLayer 的内部结构，否则就会打断 forward 流程：
# class BartEncoderLayer(nn.Module):  # 🚫 拆分它就会坏
#     ...
# 总结：
# _no_split_modules 是用来保护关键模块（如 Encoder/Decoder Layer）在结构变换时不被“打断”。
# 它只在 分布式训练、加载大模型分片、导出 ONNX 等场景中会被使用。
# 平常用 model = BartModel.from_pretrained(...) 是不会触发的。除非你启用了大模型加载、FSDP、DeepSpeed 等。

In [51]:
aa=torch.tensor([[0, 6, 10, 4, 2], [0, 8, 12, 2,0]])

In [52]:
aa

tensor([[ 0,  6, 10,  4,  2],
        [ 0,  8, 12,  2,  0]])

In [53]:
aa.ne(0)

tensor([[False,  True,  True,  True,  True],
        [False,  True,  True,  True, False]])

In [54]:
# ⚙️ __init_subclass__() 的作用：
# 这个是 Python 类机制的一个钩子方法，当有其他类继承这个类时会触发。这里用它来发出弃用警告：
# 也就是说，只要用户写了：class MyModel(PretrainedBartModel): ...
# 就会看到警告：“请用 BartPreTrainedModel”。
# 这段代码是为了兼容旧代码和拼写错误的类名，统一迁移到正确的 BartPreTrainedModel，并通过 
# __init_subclass__() 发出弃用警告。
class PretrainedBartModel(BartPreTrainedModel): # ✅ 旧的类名
    def __init_subclass__(self):
        warnings.warn(
            "The class `PretrainedBartModel` has been depreciated, please use `BartPreTrainedModel` instead.",
            FutureWarning,
        )
# 这是为了兼容拼错类名的情况。有人可能写错了：
# class MyModel(BartPretrainedModel): ...
# 为了不直接报错，而是发出友好的警告提示：
# 请使用正确的 BartPreTrainedModel
# BartPretrainedModel 里面train是小写的t,这里防止的是拼错的情况
# BartPreTrainedModel Trained是大写的T
class BartPretrainedModel(BartPreTrainedModel):  # ❌ 这个是拼错的类名
    def __init_subclass__(self):
        warnings.warn(
            "The class `PretrainedBartModel` has been depreciated, please use `BartPreTrainedModel` instead.",
            FutureWarning,
        )

In [ ]:
# __init_subclass__ 是 Python 提供的一个钩子方法（hook），在一个类被 继承 时自动调用，用于在
# 定义子类时做一些处理，而不是在实例化时调用。
# class Base:
#     def __init_subclass__(cls):
#         print(f"{cls.__name__} is a subclass of Base")

# class Sub(Base):
#     pass  # 触发 __init_subclass__，会打印信息
# 当你定义 Sub(Base) 这个类时，Base.__init_subclass__ 会被自动调用，并传入 cls=Sub。
# ✅ 用法场景
# 给子类添加默认行为或注册信息
# 发出警告（如你看到的例子）
# 校验子类是否实现了某些方法

In [ ]:
# 本质上在做什么？
# 这是为了兼容历史代码中的旧类名或拼错的类名：

# 类名	用途
# PretrainedBartModel	早期版本遗留下来的类，已弃用
# BartPretrainedModel	用户可能拼写错误的类名，也予以警告处理
# BartPreTrainedModel	✅ 当前推荐使用的类名（注意大小写）

In [55]:
BART_START_DOCSTRING = r"""
    This model inherits from [`PreTrainedModel`]. Check the superclass documentation for the generic methods the
    library implements for all its model (such as downloading or saving, resizing the input embeddings, pruning heads
    etc.)

    This model is also a PyTorch [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) subclass.
    Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage
    and behavior.

    Parameters:
        config ([`BartConfig`]):
            Model configuration class with all the parameters of the model. Initializing with a config file does not
            load the weights associated with the model, only the configuration. Check out the
            [`~PreTrainedModel.from_pretrained`] method to load the model weights.
"""

BART_GENERATION_EXAMPLE = r"""
    Summarization example:

    ```python
    >>> from transformers import AutoTokenizer, BartForConditionalGeneration

    >>> model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
    >>> tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

    >>> ARTICLE_TO_SUMMARIZE = (
    ...     "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    ...     "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    ...     "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
    ... )
    >>> inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors="pt")

    >>> # Generate Summary
    >>> summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=20)
    >>> tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    'PG&E scheduled the blackouts in response to forecasts for high winds amid dry conditions'
    ```

    Mask filling example:

    ```python
    >>> from transformers import AutoTokenizer, BartForConditionalGeneration

    >>> tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
    >>> model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

    >>> TXT = "My friends are <mask> but they eat too many carbs."
    >>> input_ids = tokenizer([TXT], return_tensors="pt")["input_ids"]
    >>> logits = model(input_ids).logits

    >>> masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
    >>> probs = logits[0, masked_index].softmax(dim=0)
    >>> values, predictions = probs.topk(5)

    >>> tokenizer.decode(predictions).split()
    ['not', 'good', 'healthy', 'great', 'very']
    ```
"""

BART_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
            Indices of input sequence tokens in the vocabulary. Padding will be ignored by default should you provide
            it.

            Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.

            [What are input IDs?](../glossary#input-ids)
        attention_mask (`torch.Tensor` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.

            [What are attention masks?](../glossary#attention-mask)
        decoder_input_ids (`torch.LongTensor` of shape `(batch_size, target_sequence_length)`, *optional*):
            Indices of decoder input sequence tokens in the vocabulary.

            Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.

            [What are decoder input IDs?](../glossary#decoder-input-ids)

            Bart uses the `eos_token_id` as the starting token for `decoder_input_ids` generation. If `past_key_values`
            is used, optionally only the last `decoder_input_ids` have to be input (see `past_key_values`).

            For translation and summarization training, `decoder_input_ids` should be provided. If no
            `decoder_input_ids` is provided, the model will create this tensor by shifting the `input_ids` to the right
            for denoising pre-training following the paper.
        decoder_attention_mask (`torch.LongTensor` of shape `(batch_size, target_sequence_length)`, *optional*):
            Default behavior: generate a tensor that ignores pad tokens in `decoder_input_ids`. Causal mask will also
            be used by default.

            If you want to change padding behavior, you should read [`modeling_bart._prepare_decoder_attention_mask`]
            and modify to your needs. See diagram 1 in [the paper](https://arxiv.org/abs/1910.13461) for more
            information on the default strategy.
        head_mask (`torch.Tensor` of shape `(encoder_layers, encoder_attention_heads)`, *optional*):
            Mask to nullify selected heads of the attention modules in the encoder. Mask values selected in `[0, 1]`:

            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.

        decoder_head_mask (`torch.Tensor` of shape `(decoder_layers, decoder_attention_heads)`, *optional*):
            Mask to nullify selected heads of the attention modules in the decoder. Mask values selected in `[0, 1]`:

            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.

        cross_attn_head_mask (`torch.Tensor` of shape `(decoder_layers, decoder_attention_heads)`, *optional*):
            Mask to nullify selected heads of the cross-attention modules in the decoder. Mask values selected in `[0,
            1]`:

            - 1 indicates the head is **not masked**,
            - 0 indicates the head is **masked**.

        encoder_outputs (`tuple(tuple(torch.FloatTensor)`, *optional*):
            Tuple consists of (`last_hidden_state`, *optional*: `hidden_states`, *optional*: `attentions`)
            `last_hidden_state` of shape `(batch_size, sequence_length, hidden_size)`, *optional*) is a sequence of
            hidden-states at the output of the last layer of the encoder. Used in the cross-attention of the decoder.
        past_key_values (`tuple(tuple(torch.FloatTensor))`, *optional*, returned when `use_cache=True` is passed or when `config.use_cache=True`):
            Tuple of `tuple(torch.FloatTensor)` of length `config.n_layers`, with each tuple having 2 tensors of shape
            `(batch_size, num_heads, sequence_length, embed_size_per_head)`) and 2 additional tensors of shape
            `(batch_size, num_heads, encoder_sequence_length, embed_size_per_head)`.

            Contains pre-computed hidden-states (key and values in the self-attention blocks and in the cross-attention
            blocks) that can be used (see `past_key_values` input) to speed up sequential decoding.

            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`.
        inputs_embeds (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
            Optionally, instead of passing `input_ids` you can choose to directly pass an embedded representation.
            This is useful if you want more control over how to convert `input_ids` indices into associated vectors
            than the model's internal embedding lookup matrix.
        decoder_inputs_embeds (`torch.FloatTensor` of shape `(batch_size, target_sequence_length, hidden_size)`, *optional*):
            Optionally, instead of passing `decoder_input_ids` you can choose to directly pass an embedded
            representation. If `past_key_values` is used, optionally only the last `decoder_inputs_embeds` have to be
            input (see `past_key_values`). This is useful if you want more control over how to convert
            `decoder_input_ids` indices into associated vectors than the model's internal embedding lookup matrix.

            If `decoder_input_ids` and `decoder_inputs_embeds` are both unset, `decoder_inputs_embeds` takes the value
            of `inputs_embeds`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        output_attentions (`bool`, *optional*):
            Whether or not to return the attentions tensors of all attention layers. See `attentions` under returned
            tensors for more detail.
        output_hidden_states (`bool`, *optional*):
            Whether or not to return the hidden states of all layers. See `hidden_states` under returned tensors for
            more detail.
        return_dict (`bool`, *optional*):
            Whether or not to return a [`~utils.ModelOutput`] instead of a plain tuple.
"""

In [56]:
config.max_position_embeddings

1024

In [57]:
math.sqrt(2)

1.4142135623730951

In [58]:
aa=torch.randn((2,8,32))

In [ ]:
aa

In [60]:
aa[:, :, -1] # 只是选取最后一个维度的32个元素的向量中,最后那个数据元素

tensor([[ 0.9493, -1.3439, -0.1623,  1.3046,  0.6839,  1.0765, -0.0886,  0.5509],
        [ 0.8409,  0.5754,  0.5625, -0.6599,  1.9180, -1.1001,  0.2933,  0.5442]])

In [61]:
bb=torch.Tensor([[1,1,0],[0,5,6]])
0 in bb

True

In [62]:
torch.rand([])

tensor(0.6346)

In [63]:
config.encoder_layerdrop

0.0

In [64]:
# BART 编码器模块，由多个 BartEncoderLayer 层堆叠组成。
class BartEncoder(BartPreTrainedModel):

    def __init__(self, config: BartConfig, embed_tokens: Optional[nn.Embedding] = None):
        super().__init__(config)
        self.dropout = config.dropout # dropout 概率
        self.layerdrop = config.encoder_layerdrop # layerdrop 概率，用于训练时跳过层
        embed_dim = config.d_model # 嵌入维度
        self.padding_idx = config.pad_token_id # 填充id
        self.max_source_positions = config.max_position_embeddings # 最大位置序列长度
        embed_scale = math.sqrt(embed_dim) if config.scale_embedding else 1.0  # 嵌入缩放
        # token嵌入
        self.embed_tokens = BartScaledWordEmbedding(
            config.vocab_size, embed_dim, self.padding_idx, embed_scale=embed_scale
        )
        # 如果有指定的嵌入权重,就设置为传入的,这是有预训练词向量的情况
        if embed_tokens is not None:
            self.embed_tokens.weight = embed_tokens.weight
        # 使用学习的位置编码
        self.embed_positions = BartLearnedPositionalEmbedding(
            config.max_position_embeddings,
            embed_dim,
        )
        # 构建 N 层编码器
        self.layers = nn.ModuleList([BartEncoderLayer(config) for _ in range(config.encoder_layers)])
        # 是否使用 FlashAttention 或 SDPA 优化 attention 计算
        self._use_flash_attention_2 = config._attn_implementation == "flash_attention_2"
        self._use_sdpa = config._attn_implementation == "sdpa"
        self.layernorm_embedding = nn.LayerNorm(embed_dim)  # 输入归一化层

        self.gradient_checkpointing = False   # 控制是否使用梯度检查点以节省显存
        self.post_init()  # 初始化权重
    # 获取词嵌入
    def get_input_embeddings(self):
        return self.embed_tokens
    # 设置词嵌入
    def set_input_embeddings(self, value):
        self.embed_tokens = value

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, BaseModelOutput]:
        # 参数优先级：函数输入 > 配置文件
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict # 是否返回字典结构

         # input_ids 与 inputs_embeds 只能二选一
        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None: # 如果只是设定了input_ids
            input = input_ids
            input_ids = input_ids.view(-1, input_ids.shape[-1])  # reshape to 2D
        elif inputs_embeds is not None: # 只是设定了inputs_embeds
            input = inputs_embeds[:, :, -1]  # 获取伪位置输入用于位置编码
        else: # 如果两个都没有指定,报错
            raise ValueError("You have to specify either input_ids or inputs_embeds")
        # 如果还没有对token嵌入表示
        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens(input_ids) # 就嵌入

        embed_pos = self.embed_positions(input) # 为位置嵌入,每个位置也对应一个向量表示
        embed_pos = embed_pos.to(inputs_embeds.device)
        # 初始输入 = token_embed + pos_embed
        hidden_states = inputs_embeds + embed_pos
        hidden_states = self.layernorm_embedding(hidden_states) # 层标准化
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training) # dropout

        # 构建注意力 mask
        if attention_mask is not None:
            if self._use_flash_attention_2: # 如果是使用flash注意力
                attention_mask = attention_mask if 0 in attention_mask else None  # FlashAttention 中 0 视为需屏蔽
            # 如果是使用sdpa 这时就不能有head_mask,也不能指定输出注意力权重
            elif self._use_sdpa and head_mask is None and not output_attentions:
                # 当使用 SDPA 时，如果不满足条件,会退回到手动实现方式，在所有情况下都需要一个 4D 的因果掩码。
                # [bsz, seq_len] -> [bsz, 1, tgt_seq_len, src_seq_len]
                attention_mask = _prepare_4d_attention_mask_for_sdpa(attention_mask, inputs_embeds.dtype)
            else: # 这种是eager 手动形式的注意力
                # [bsz, seq_len] -> [bsz, 1, tgt_seq_len, src_seq_len]
                attention_mask = _prepare_4d_attention_mask(attention_mask, inputs_embeds.dtype)

        encoder_states = () if output_hidden_states else None # 每层的编码器输出
        all_attentions = () if output_attentions else None

        # 检查 head_mask 长度合法性
        if head_mask is not None:
            if head_mask.size()[0] != (len(self.layers)): # head mask的最外轴的大小必须是所有的层数
                raise ValueError( 
                    f"The head_mask should be specified for {len(self.layers)} layers, but it is for"
                    f" {head_mask.size()[0]}."
                )
        # 编码器层前向过程
        for idx, encoder_layer in enumerate(self.layers): # 遍历每一层
            if output_hidden_states: # 如果指定要输出每一层的编码器输出
                encoder_states = encoder_states + (hidden_states,) # 第一个是词嵌入
            # add LayerDrop (see https://arxiv.org/abs/1909.11556 for description)
            # 层级 Dropout（LayerDrop），训练时随机跳过某层
            to_drop = False
            if self.training: # 如果是训练模式
                dropout_probability = torch.rand([]) # 随机的一个dropout概率
                # 如果随机到的数字<配置中指定的layerdrop
                if dropout_probability < self.layerdrop:  
                    to_drop = True # 这时设置to_drop为True
            # if to_drop: 分支中的逻辑 确实是跳过了当前编码器层的前向计算，这是实现 LayerDrop 的核心。
            # 此处 to_drop 是基于 self.layerdrop 的概率随机决定的（仅在训练时生效），如果为 True，则当前这层 
            # encoder layer 会被跳过，不进行前向传播。这种技术称为 LayerDrop，类似 Dropout，但作用在整个层级结
            # 构上，用于正则化深层模型。跳过后当前 hidden_states 会直接传给下一层。
            if to_drop: # 如果to_drop为True
                layer_outputs = (None, None) # 跳过该层计算，保持 hidden_states 不变
            else: # 如果to_drop=False
                if self.gradient_checkpointing and self.training: # 如果使用梯度检查,并且是训练模式
                    # 用 gradient checkpoint 节省显存，代价是多次前向
                    layer_outputs = self._gradient_checkpointing_func(
                        encoder_layer.__call__,
                        hidden_states,
                        attention_mask,
                        (head_mask[idx] if head_mask is not None else None), # 使用当前层的head_mask
                        output_attentions,
                    )
                else: # 如果是评估模式或者禁用了梯度检查 # 正常前向传播
                    layer_outputs = encoder_layer(
                        hidden_states,
                        attention_mask,
                        layer_head_mask=(head_mask[idx] if head_mask is not None else None),
                        output_attentions=output_attentions,
                    )

                hidden_states = layer_outputs[0] # 更新隐藏状态,供下个迭代使用
            # 如果要输出注意力矩阵
            if output_attentions:
                all_attentions = all_attentions + (layer_outputs[1],)
        # 这个是最后一层的编码器输出状态
        if output_hidden_states:
            encoder_states = encoder_states + (hidden_states,)
        # 根据情况返回元组或结构化对象(字典结构)
        if not return_dict:
            return tuple(v for v in [hidden_states, encoder_states, all_attentions] if v is not None)
        return BaseModelOutput(
            last_hidden_state=hidden_states, hidden_states=encoder_states, attentions=all_attentions
        )

In [65]:
# BART 解码器模块，由多个 BartDecoderLayer 层组成（Transformer 结构的解码器部分）。
class BartDecoder(BartPreTrainedModel):
    
    def __init__(self, config: BartConfig, embed_tokens: Optional[nn.Embedding] = None):
        super().__init__(config)
        self.dropout = config.dropout  # dropout 概率
        # layerdrop 是一种正则手段，在训练时以一定概率跳过整个 decoder layer
        self.layerdrop = config.decoder_layerdrop
        self.padding_idx = config.pad_token_id # padding token 的 id，用于 mask 掉 padding 部分
        # 最大解码长度（即位置编码的最大长度）
        self.max_target_positions = config.max_position_embeddings
        # 是否使用缩放嵌入（通常 sqrt(d_model)），以控制初始嵌入的方差，避免太大或太小
        embed_scale = math.sqrt(config.d_model) if config.scale_embedding else 1.0
        # 词嵌入层，实际为可学习参数表，输出 shape 为 [bsz, seq_len, d_model]
        self.embed_tokens = BartScaledWordEmbedding(
            config.vocab_size, config.d_model, self.padding_idx, embed_scale=embed_scale
        )
        # 如果外部传入 embedding，则使用共享 embedding（通常 encoder 与 decoder 共享词向量）
        if embed_tokens is not None:
            self.embed_tokens.weight = embed_tokens.weight
        # 位置嵌入，LearnedPositionalEmbedding 是可学习的位置编码（相比于 sinusoidal 编码）
        self.embed_positions = BartLearnedPositionalEmbedding(
            config.max_position_embeddings,
            config.d_model,
        )
        # 构建 N 层解码器，每层是一个标准的 BartDecoderLayer
        self.layers = nn.ModuleList([BartDecoderLayer(config) for _ in range(config.decoder_layers)])
         # 是否使用 Flash Attention v2（高效注意力实现，适用于较长序列）
        self._use_flash_attention_2 = config._attn_implementation == "flash_attention_2"
         # 是否使用 SDPA（torch.nn.functional.scaled_dot_product_attention），为 PyTorch 原生高效实现
        self._use_sdpa = config._attn_implementation == "sdpa"
         # 输入嵌入后进行 layer norm，有助于稳定训练过程
        self.layernorm_embedding = nn.LayerNorm(config.d_model)
        # 是否启用 gradient checkpointing（以计算图为单位进行中间层保存，节省显存）
        self.gradient_checkpointing = False
        # 初始化权重
        self.post_init()
    # 获取输入的词嵌入（用于生成时 tie-weights）
    def get_input_embeddings(self):
        return self.embed_tokens
    # 设置输入嵌入（通常是为了权重共享）
    def set_input_embeddings(self, value):
        self.embed_tokens = value

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, BaseModelOutputWithPastAndCrossAttentions]:
        # 设置默认值（可被 forward 的入参覆盖）
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # 检查输入，input_ids 与 inputs_embeds 二选一
        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both decoder_input_ids and decoder_inputs_embeds at the same time")
        elif input_ids is not None:
            input = input_ids
            input_shape = input.shape
            input_ids = input_ids.view(-1, input_shape[-1])
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1] # (b,s)
            input = inputs_embeds[:, :, -1]
        else:
            raise ValueError("You have to specify either decoder_input_ids or decoder_inputs_embeds")

        # 如果使用 past_key_values（推理时加速用，只需输入新 token）
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0
        # 如果没提供 inputs_embeds，用 embedding 层查表
        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens(input)
        # 因果注意力 mask 构建（不同模式处理不同）
        # 这段代码的作用就是构建因果注意力掩码（causal attention mask），并根据不同情况选择合适的构建方式。
        # 以下是详细注释版代码，包含设计意图和功能说明，全部写在代码内部：
        if self._use_flash_attention_2:
            # flash attention 不支持全 1 mask，因此仅在存在 padding 时保留 mask
            # Flash Attention 要求不传入全为 1 的 attention_mask（即没有 padding），
            # 否则会报错，因此仅在存在 padding（即 attention_mask 中存在 0）时才保留 mask。
            attention_mask = attention_mask if (attention_mask is not None and 0 in attention_mask) else None
        elif self._use_sdpa and not output_attentions and cross_attn_head_mask is None:
            # 使用 SDPA 且不返回注意力，构建 4D mask（适配 scaled_dot_product_attention）
            # - 不需要输出注意力权重（output_attentions=False）
            # - 不存在 cross attention 的 head mask（cross_attn_head_mask=None）
            # 满足这些条件时，可以使用 SDPA 提供的更高效实现
            # 构建的是 4D 版本的因果掩码（[batch_size, num_heads, query_len, key_len]），
            # 用于告诉模型：每个位置只能看到当前和前面的 token。
            attention_mask = _prepare_4d_causal_attention_mask_for_sdpa(
                attention_mask,
                input_shape,
                inputs_embeds,
                past_key_values_length,
            )
        else:
            # 常规模式构建 4D causal mask（decoder 只能看自己和之前的 token）
            # 非 Flash Attention 或 SDPA 情况（比如开启了 output_attentions 或使用了 head mask），
            # 使用常规方法构建 4D 因果掩码，限制 decoder 每个 token 只能看到它自己和前面的 token。
            attention_mask = _prepare_4d_causal_attention_mask(
                attention_mask, input_shape, inputs_embeds, past_key_values_length
            )

        # 如果有 encoder_hidden_states，这时是交叉注意力,这里构建编码器填充掩码用于交叉注意力
        # 该段代码是在交叉注意力阶段（cross-attention）构建用于编码器输出的 attention mask
        if encoder_hidden_states is not None and encoder_attention_mask is not None:
            if self._use_flash_attention_2:
                # Flash Attention v2 要求 attention_mask 不能是全 1（即没有 padding），否则会出错。
                # 因此仅在存在 padding（即 mask 中有 0）时才保留掩码。
                encoder_attention_mask = encoder_attention_mask if 0 in encoder_attention_mask else None
            elif self._use_sdpa and cross_attn_head_mask is None and not output_attentions:
                # 上面的条件是使用sdpa,使用的话头掩码就不能有,而且不能输出注意力权重矩阵
                # [bsz, seq_len] -> [bsz, 1, tgt_seq_len, src_seq_len]
                # 从形状为 2D 的掩码创建形状为 `(batch_size, 1, query_length, key_value_length)` 的非因果 4D 掩码
                # - 不使用 head 掩码（cross_attn_head_mask 为 None）
                # - 不输出注意力权重（output_attentions=False）
                # 满足这些条件才能走 SDPA 的高效实现。
                # 将 2D mask [batch_size, src_len] 转为 4D mask [batch_size, 1, tgt_len, src_len]
                # 这是一个 **非因果掩码**，用于 cross-attention 阶段的编码器掩码。
                # 表示 decoder 的每个 query 可以看到 encoder 的所有 key（除非 key 是 padding）
                encoder_attention_mask = _prepare_4d_attention_mask_for_sdpa(
                    encoder_attention_mask,
                    inputs_embeds.dtype,
                    tgt_len=input_shape[-1],  # tgt_len = decoder query 的长度
                )
            else:
                # 使用普通方式构建 cross-attention 的编码器填充掩码
                # 同样将 2D mask 扩展为 4D 形状 [batch_size, 1, tgt_len, src_len]
                encoder_attention_mask = _prepare_4d_attention_mask(
                    encoder_attention_mask, inputs_embeds.dtype, tgt_len=input_shape[-1]
                )

         # 位置嵌入（支持 past_key_values 时从中间开始编码）
        positions = self.embed_positions(input, past_key_values_length)
        positions = positions.to(inputs_embeds.device)
        # 加上位置嵌入后 layernorm，然后 dropout
        hidden_states = inputs_embeds + positions
        hidden_states = self.layernorm_embedding(hidden_states)
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)
        # 若使用 gradient checkpointing，禁用 cache（两者冲突）
        if self.gradient_checkpointing and self.training:
            if use_cache:
                logger.warning_once(
                    "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                )
                use_cache = False

        # 初始化返回值缓存（仅在用户需要时收集）
        all_hidden_states = () if output_hidden_states else None # 是否输出每一层的解码器输出
        all_self_attns = () if output_attentions else None # 所有的注意力权重矩阵
        # 所有的交叉注意力权重矩阵,encoder_hidden_states is not None 是说有传入编码器输出
        all_cross_attentions = () if (output_attentions and encoder_hidden_states is not None) else None
        next_decoder_cache = () if use_cache else None # 缓存的past_key_value

        # 检查 head_mask / cross_attn_head_mask 是否具有正确的层数
        # head_mask 用于控制每一层的 self-attention 中哪些头参与计算
        # cross_attn_head_mask 用于控制每一层的 cross-attention 中哪些头参与计算
        # 若指定了，则必须与 decoder 层数一致
        for attn_mask, mask_name in zip([head_mask, cross_attn_head_mask], ["head_mask", "cross_attn_head_mask"]):
            if attn_mask is not None:
                if attn_mask.size()[0] != (len(self.layers)):
                    raise ValueError(
                        f"The `{mask_name}` should be specified for {len(self.layers)} layers, but it is for"
                        f" {head_mask.size()[0]}."
                    )
        # 遍历 decoder 的每一层进行前向传播
        for idx, decoder_layer in enumerate(self.layers):
            # 若启用输出中间层 hidden states，则保存当前层输入
            if output_hidden_states:
                all_hidden_states += (hidden_states,)
            # LayerDrop：训练时以一定概率跳过某一层，以提升模型鲁棒性
            if self.training:
                dropout_probability = torch.rand([])  # 生成一个标量随机数
                if dropout_probability < self.layerdrop: 
                    continue # 跳过当前层，直接进入下一层
            # 获取本层的 past_key_value（用于加速生成时的缓存）
            past_key_value = past_key_values[idx] if past_key_values is not None else None
             # 若启用梯度检查点（节省显存），则通过 checkpointing 执行当前 decoder layer 的前向计算
            if self.gradient_checkpointing and self.training:
                layer_outputs = self._gradient_checkpointing_func(
                    decoder_layer.__call__,  # 被 checkpoint 的函数
                    hidden_states,  # 当前输入
                    attention_mask, # decoder 的 self-attention mask（通常为 causal mask）
                    encoder_hidden_states,  # cross attention 中的 key/value（来自 encoder）
                    encoder_attention_mask,  # cross attention 的 attention mask（通常是 padding mask）
                    head_mask[idx] if head_mask is not None else None, # 指定当前层的自注意力的 head_mask
                    cross_attn_head_mask[idx] if cross_attn_head_mask is not None else None, # cross-attention 的头掩码
                    None, # position bias，旧版支持，现在未使用
                    output_attentions,  # 是否输出注意力权重
                    use_cache,   # 是否使用缓存（加速生成）
                )
            else: # 标准前向传播（无 gradient checkpointing）
                layer_outputs = decoder_layer(
                    hidden_states,
                    attention_mask=attention_mask,
                    encoder_hidden_states=encoder_hidden_states,
                    encoder_attention_mask=encoder_attention_mask,
                    layer_head_mask=(head_mask[idx] if head_mask is not None else None),
                    cross_attn_layer_head_mask=(
                        cross_attn_head_mask[idx] if cross_attn_head_mask is not None else None
                    ),
                    past_key_value=past_key_value,
                    output_attentions=output_attentions,
                    use_cache=use_cache,
                )
            hidden_states = layer_outputs[0] # 更新 hidden_states 供下一层使用
             # 如果开启 use_cache，将当前层的 past_key_value 加入缓存中，供生成用
            if use_cache:
                next_decoder_cache += (layer_outputs[3 if output_attentions else 1],)
            # 若开启输出 attentions，则收集当前层的 self-attention 权重
            if output_attentions:
                all_self_attns += (layer_outputs[1],)
                # 如果存在 编码器输出（即为 cross-attention），则收集 cross-attention 权重
                if encoder_hidden_states is not None:
                    all_cross_attentions += (layer_outputs[2],)

        # 最后一层的 hidden_states 也加入中间层输出
        if output_hidden_states:
            all_hidden_states += (hidden_states,)
        # 最终决定是否返回缓存
        next_cache = next_decoder_cache if use_cache else None
        # 若不要求返回 dict，则返回 tuple 形式（为了兼容老接口）
        if not return_dict:
            return tuple(
                v
                for v in [hidden_states, next_cache, all_hidden_states, all_self_attns, all_cross_attentions]
                if v is not None
            )
        # 默认返回 BaseModelOutputWithPastAndCrossAttentions，结构化封装全部输出
        return BaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states,  # decoder 最后一层输出
            past_key_values=next_cache,   # 各层缓存（用于生成加速）
            hidden_states=all_hidden_states,   # 所有中间层 hidden state（如果启用）
            attentions=all_self_attns,  # 所有 self-attention 权重（如果启用）
            cross_attentions=all_cross_attentions,  # 所有 cross-attention 权重（如果启用）
        )

In [66]:
config.tie_word_embeddings

True

In [ ]:
# 在 BartEncoder 和 BartDecoder 的初始化中，会将 self.shared 这个共享的嵌入层传进去。所以理论上它们内部的 embed_tokens 
# 都会指向 self.shared。

# 但有些模型 checkpoint（例如 facebook/bart-large-cnn）中，实际保存的是两个独立的权重矩阵（decoder 有自己的嵌入），因此加载后：

# self.encoder.embed_tokens 和 self.decoder.embed_tokens 是两个不同对象；

# 此时模型内部就需要手动调用 _tie_or_clone_weights 来确保它们的 weight 是同一个张量。

In [68]:
@add_start_docstrings(
    "The bare BART Model outputting raw hidden-states without any specific head on top.",
    BART_START_DOCSTRING,
)
class BartModel(BartPreTrainedModel):
    # 指定共享参数的键（用于词嵌入权重的参数共享）
    _tied_weights_keys = ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight"]

    def __init__(self, config: BartConfig):
        super().__init__(config)
        # 提取配置中的 pad token id 和词表大小
        padding_idx, vocab_size = config.pad_token_id, config.vocab_size
        # 如果启用 scale_embedding，则将嵌入放大 sqrt(d_model)，否则为 1
        embed_scale = math.sqrt(config.d_model) if config.scale_embedding else 1.0
        # 创建共享嵌入层，encoder 和 decoder 共用
        self.shared = BartScaledWordEmbedding(vocab_size, config.d_model, padding_idx, embed_scale=embed_scale)
         # 初始化编码器和解码器，传入共享词嵌入
        self.encoder = BartEncoder(config, self.shared)
        self.decoder = BartDecoder(config, self.shared)

        # 初始化所有权重
        self.post_init()

    def _tie_weights(self):
        # 如果配置中启用了词嵌入共享（tie_word_embeddings=True），则需要将 encoder、decoder 和 shared 的词嵌入层权重绑定为同一个张量。
        # 这样做的目的是减少模型参数量，同时提升 encoder 和 decoder 在词嵌入层的一致性（如在翻译任务中共享词汇表）。
        if self.config.tie_word_embeddings:
            # 特殊处理 “meta” 设备的情况：
            # 某些模型（例如 "facebook/bart-large-cnn"）在加载权重时，其 shared 层可能处于未实际分配内存的 “meta” 设备上
            # （即 lazy weight loading），
            # 而 decoder 的 embed_tokens 已被实际分配到设备上了（如 CPU/GPU）。
            # 此时不能直接使用 shared，因为它还没有实际的内存张量。
            # 所以我们优先将 encoder 的嵌入层权重与 decoder 的绑定，再将 shared 的也与 decoder 的绑定（
            # 使得三者共享同一权重张量）。
            if self.shared.weight.device == torch.device(
                "meta") and self.decoder.embed_tokens.weight.device != torch.device("meta"):
                # self.encoder.embed_tokens=self.decoder.embed_tokens
                self._tie_or_clone_weights(self.encoder.embed_tokens, self.decoder.embed_tokens)
                # self.shared=self.decoder.embed_tokens
                self._tie_or_clone_weights(self.shared, self.decoder.embed_tokens)
            # 正常情况（所有权重都已实际加载，且设备一致），直接将 encoder 和 decoder 的嵌入层权重绑定到 shared 上，
            # 这样三者都共享 shared.weight 的张量（即 encoder.embed_tokens.weight 和 decoder.embed_tokens.weight 
            # 都指向 shared.weight）
            else:
                # 这里把self.encoder.embed_tokens和self.decoder.embed_tokens都设置成了self.shared
                self._tie_or_clone_weights(self.encoder.embed_tokens, self.shared)
                self._tie_or_clone_weights(self.decoder.embed_tokens, self.shared)
    # 返回模型输入嵌入层（共享词嵌入）
    def get_input_embeddings(self):
        return self.shared
     # 设置共享词嵌入（确保 encoder/decoder 同步）
    def set_input_embeddings(self, value):
        self.shared = value
        self.encoder.embed_tokens = self.shared
        self.decoder.embed_tokens = self.shared
     # 返回 encoder 子模块（可用于抽取特征）
    def get_encoder(self):
        return self.encoder
    # 返回 decoder 子模块（可用于解码）
    def get_decoder(self):
        return self.decoder

    @add_start_docstrings_to_model_forward(BART_INPUTS_DOCSTRING)
    @add_code_sample_docstrings(
        checkpoint=_CHECKPOINT_FOR_DOC,
        output_type=Seq2SeqModelOutput,
        config_class=_CONFIG_FOR_DOC,
        expected_output=_EXPECTED_OUTPUT_SHAPE,
    )
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, Seq2SeqModelOutput]:
         # 如果 decoder_input_ids 和 decoder_inputs_embeds 都没有提供，
        # 默认用 input_ids 的右移版本初始化 decoder_input_ids（符合自回归解码器输入规范）
        if decoder_input_ids is None and decoder_inputs_embeds is None:
            if input_ids is None:
                raise ValueError(
                    "If no `decoder_input_ids` or `decoder_inputs_embeds` are "
                    "passed, `input_ids` cannot be `None`. Please pass either "
                    "`input_ids` or `decoder_input_ids` or `decoder_inputs_embeds`."
                )

            decoder_input_ids = shift_tokens_right( # 解码器输入[:,:-1]
                input_ids, self.config.pad_token_id, self.config.decoder_start_token_id
            )
        # 使用配置中默认值更新控制参数（允许用户显式传入覆盖配置）
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
         # 如果不存在编码器的输出,就先调用encoeer获取编码器输出
        if encoder_outputs is None:
            encoder_outputs = self.encoder(
                input_ids=input_ids, # 编码器输入序列的 token id
                attention_mask=attention_mask,  # 编码器输入的注意力掩码，屏蔽 padding 位置
                head_mask=head_mask,  # 编码器自注意力头的掩码（可选）
                inputs_embeds=inputs_embeds,  # 编码器输入的嵌入向量（可选）
                output_attentions=output_attentions, # 是否输出注意力权重
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,    # 是否返回 dict 结构
            )
        # 如果外部传入的 encoder_outputs 是 tuple，但要求返回 dict，则包装成 BaseModelOutput
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],  # 编码器最后一层隐藏状态（主输出）
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,  # 编码器各层隐藏状态序列（可选）
                attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None, # 编码器各层注意力权重（可选）
            )

        # 解码器执行，传入编码器输出作为 cross-attention 的输入
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,  # 解码器输入的 token id（目标序列）
            attention_mask=decoder_attention_mask,  # 解码器自注意力掩码（因果掩码+padding掩码）
            encoder_hidden_states=encoder_outputs[0],# 编码器最后一层隐藏状态，作为 cross-attention 的 key/value
            encoder_attention_mask=attention_mask, # 编码器的注意力掩码，防止对 padding 位置关注
            head_mask=decoder_head_mask,   # 解码器自注意力头的掩码（可选）
            cross_attn_head_mask=cross_attn_head_mask,   # 解码器交叉注意力头的掩码（可选）
            past_key_values=past_key_values,   # 解码器缓存的历史 key/value（用于加速推理）
            inputs_embeds=decoder_inputs_embeds,   # 解码器输入的嵌入向量（可选）
            use_cache=use_cache,   # 是否启用缓存机制，提升解码效率
            output_attentions=output_attentions,   # 是否输出注意力权重
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,   # 是否返回 dict 结构
        )
        # 如果不需要返回 dict，则将解码器输出和编码器输出拼接成元组返回
        if not return_dict:
            return decoder_outputs + encoder_outputs
        # 否则,返回结构化输出
        # 标准输出结构，便于使用者访问各组件（编码器/解码器的隐藏状态、注意力等）
        return Seq2SeqModelOutput(
            last_hidden_state=decoder_outputs.last_hidden_state,   # 解码器最后一层隐藏状态（主输出）
            past_key_values=decoder_outputs.past_key_values,  # 解码器缓存的历史 key/value
            decoder_hidden_states=decoder_outputs.hidden_states,  # 解码器各层隐藏状态序列（可选）
            decoder_attentions=decoder_outputs.attentions,  # 解码器各层自注意力权重（可选）
            cross_attentions=decoder_outputs.cross_attentions,  # 解码器各层交叉注意力权重（可选）
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,  # 编码器最后一层隐藏状态
            encoder_hidden_states=encoder_outputs.hidden_states,    # 编码器各层隐藏状态序列（可选）
            encoder_attentions=encoder_outputs.attentions,    # 编码器各层注意力权重（可选）
        )

In [ ]:
help(BartModel.forward)

In [ ]:
 # Example:
    
 #    ```python
 #    >>> from transformers import AutoTokenizer, BartModel
 #    >>> import torch
    
 #    >>> tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
 #    >>> model = BartModel.from_pretrained("facebook/bart-base")
    
 #    >>> inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
 #    >>> outputs = model(**inputs)
    
 #    >>> last_hidden_states = outputs.last_hidden_state
    # ```

In [69]:
from transformers import AutoTokenizer, BartModel

In [70]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartModel.from_pretrained("facebook/bart-base")

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [71]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

In [72]:
inputs

{'input_ids': tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [73]:
print(type(outputs.past_key_values),len(outputs.past_key_values))

<class 'tuple'> 6


In [74]:
print(len(outputs.past_key_values[0]))

4


In [75]:
[i.shape for i in outputs.past_key_values[0]]

[torch.Size([1, 12, 8, 64]),
 torch.Size([1, 12, 8, 64]),
 torch.Size([1, 12, 8, 64]),
 torch.Size([1, 12, 8, 64])]

In [76]:
config.decoder_layers

12

In [77]:
outputs.last_hidden_state.shape

torch.Size([1, 8, 768])

In [78]:
print(outputs.decoder_hidden_states,outputs.encoder_last_hidden_state.shape)

None torch.Size([1, 8, 768])


In [ ]:
# 虽然在 __init__() 中：
# self.shared = BartScaledWordEmbedding(...)
# self.encoder = BartEncoder(config, self.shared)
# self.decoder = BartDecoder(config, self.shared)
# 已经传入了同一个 Embedding 实例 self.shared，表面上看已经实现了共享，但实际还需要 self._tie_weights() 
# 来确保权重绑定为同一个张量对象，特别是在以下几种场景中才真正“起作用”：
# _tie_weights() 作用场景详解：
# 1.处理部分 checkpoint 权重结构不一致的问题：
# 比如有些模型的权重文件中，只保存了 decoder.embed_tokens.weight，没有保存 shared.weight 或 
# encoder.embed_tokens.weight，那就需要靠 _tie_weights() 手动绑定权重关系，否则三者会不一致。
# 2.处理 Meta device 场景：
# 在 lazy init / pipeline parallelism 场景中（如加载在 meta 设备时），三个模块的 Embedding 虽然逻辑上是同一
# 个，但还没有实际分配权重张量，必须调用 _tie_weights() 来显式绑定。
# 3.适配 from_pretrained() 加载模型后：
# from_pretrained() 加载模型时可能只加载 decoder.embed_tokens 的权重，此时模型内部 self.shared 是一个新
# 的 Embedding 实例，必须通过 _tie_weights() 把它和 decoder/encoder 显式绑定为同一个张量。
# 4.模型结构定义与权重加载分离设计：
# 为了将结构定义（__init__）和权重关系（共享 or 不共享）分开设计，Transformers 框架统一使用 _tie_weights() 
# 来在模型构造之后处理“权重绑定关系”，实现更灵活的配置控制。

In [80]:
def _resize_token_embeddings(self, new_num_tokens, pad_to_multiple_of=None, mean_resizing=True):
    old_embeddings = self.get_input_embeddings()
    new_embeddings = self._get_resized_embeddings(
        old_embeddings, new_num_tokens, pad_to_multiple_of, mean_resizing
    )
    if hasattr(old_embeddings, "_hf_hook"):
        hook = old_embeddings._hf_hook
        add_hook_to_module(new_embeddings, hook)
    old_embeddings_requires_grad = old_embeddings.weight.requires_grad
    new_embeddings.requires_grad_(old_embeddings_requires_grad)
    self.set_input_embeddings(new_embeddings)
    is_quantized = hasattr(self, "hf_quantizer") and self.hf_quantizer is not None

    # Update new_num_tokens with the actual size of new_embeddings
    if pad_to_multiple_of is not None:
        if is_deepspeed_zero3_enabled() and not is_quantized:
            with deepspeed.zero.GatheredParameters(new_embeddings.weight, modifier_rank=None):
                new_num_tokens = new_embeddings.weight.shape[0]
        else:
            new_num_tokens = new_embeddings.weight.shape[0]

    # if word embeddings are not tied, make sure that lm head is resized as well
    if (
        self.get_output_embeddings() is not None
        and not self.config.get_text_config(decoder=True).tie_word_embeddings
    ):
        old_lm_head = self.get_output_embeddings()
        if isinstance(old_lm_head, torch.nn.Embedding):
            new_lm_head = self._get_resized_embeddings(
                old_lm_head, new_num_tokens, mean_resizing=mean_resizing)
        else:
            new_lm_head = self._get_resized_lm_head(
                old_lm_head, new_num_tokens, mean_resizing=mean_resizing)
        if hasattr(old_lm_head, "_hf_hook"):
            hook = old_lm_head._hf_hook
            add_hook_to_module(new_lm_head, hook)
        old_lm_head_requires_grad = old_lm_head.weight.requires_grad
        new_lm_head.requires_grad_(old_lm_head_requires_grad)
        self.set_output_embeddings(new_lm_head)

    return self.get_input_embeddings()

In [81]:
# 定义了一个子类 BartForConditionalGeneration。
# 继承自：
# BartPreTrainedModel：提供权重加载、保存、初始化等通用模型机制。
# GenerationMixin：混入生成逻辑，如 .generate() 方法，支持 beam search、greedy search 等。
@add_start_docstrings(
    "The BART Model with a language modeling head. Can be used for summarization.", BART_START_DOCSTRING
)
class BartForConditionalGeneration(BartPreTrainedModel, GenerationMixin):
    # 表示该模型包含一个子模块叫 self.model（后续代码会看到），这是原始的 BartModel，它负责编码器-解码器的主体结构。
    base_model_prefix = "model"
    # 词嵌入共享（weight tying），即：编码器词嵌入 (encoder.embed_tokens)
    # 解码器词嵌入 (decoder.embed_tokens) 最后的 lm_head（语言建模输出层）
    # 都共享同一组权重。
    _tied_weights_keys = ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight", "lm_head.weight"]
    # 加载权重时，如果缺失了这些 key，不抛出警告或错误。
    # final_logits_bias 是一个额外加在输出 logits 上的偏置项（通常初始化为0），用于更灵活地调整预测概率。
    # 不将它设置为必须项，兼容旧模型权重或不同配置下的模型加载。
    # 如果这个权重文件里没有包含 "final_logits_bias"，不会抛出错误，而是由代码中的逻辑（如 self.register_buffer(
    # ...)）自动补上默认值。
    _keys_to_ignore_on_load_missing = ["final_logits_bias"]
    # 初始化BART条件生成模型，带语言建模头。
    # 设计意图：基于BART模型，添加语言模型输出层，用于文本生成/摘要等任务。
    def __init__(self, config: BartConfig):
        super().__init__(config)
        self.model = BartModel(config) # BART基础模型，包含编码器和解码器
        # final_logits_bias是一个偏置向量，初始化为0，后续加到预测logits上，调整预测概率分布
        self.register_buffer("final_logits_bias", torch.zeros((1, self.model.shared.num_embeddings)))
        # 语言模型头，线性层，将隐藏状态映射到词表大小，用于生成token概率
        self.lm_head = nn.Linear(config.d_model, self.model.shared.num_embeddings, bias=False)
        # 权重初始化及后续处理（如tie weights）
        self.post_init()
    # 返回编码器模块，方便单独访问编码器
    def get_encoder(self):
        return self.model.get_encoder()
    # 返回解码器模块，方便单独访问解码器
    def get_decoder(self):
        return self.model.get_decoder()
    # 调整词嵌入矩阵大小（比如新增词表词汇）。
    # 同时调整 final_logits_bias 维度，保持与词表大小一致。
    def resize_token_embeddings(
        self, new_num_tokens: int, pad_to_multiple_of: Optional[int] = None, mean_resizing: bool = True
    ) -> nn.Embedding:
        # 调整嵌入矩阵大小（包括 encoder 和 decoder 的 token embedding）
        # 该方法通常在扩展词表（如添加新token）时调用
        # super() 实际调用的是 PreTrainedModel 中的 resize_token_embeddings
        new_embeddings = super().resize_token_embeddings(new_num_tokens, pad_to_multiple_of, mean_resizing)
        # 因为最终 logits 是通过 `lm_head + final_logits_bias` 得出的，
        # 所以词表大小变化后，也要同步调整 final_logits_bias 的大小，以确保维度一致
        # new_embeddings.weight.shape[0] 就是词汇表的大小，即词表中的 token 数量（vocab_size）
        self._resize_final_logits_bias(new_embeddings.weight.shape[0])
        return new_embeddings
    # 调整 final_logits_bias 的大小，确保与当前词表大小一致。
    # 如果词表变小，裁剪偏置；变大时，补0。
    def _resize_final_logits_bias(self, new_num_tokens: int) -> None:
        # 记录旧的词汇表大小
        old_num_tokens = self.final_logits_bias.shape[-1]
        # 新词汇表大小小于等于旧大小，截断 bias 以匹配新词表大小
        if new_num_tokens <= old_num_tokens:
            new_bias = self.final_logits_bias[:, :new_num_tokens]
        else: # 新词汇表大小大于旧大小，扩展 bias 矩阵，新增部分用0填充
            extra_bias = torch.zeros((
                1, new_num_tokens - old_num_tokens), device=self.final_logits_bias.device)
            new_bias = torch.cat([self.final_logits_bias, extra_bias], dim=1) # 在词汇表轴合并
        # 用新的 bias 替换旧的 final_logits_bias，注册为模型缓冲区，不作为参数训练
        self.register_buffer("final_logits_bias", new_bias)
    # 返回语言模型头，用于获取序列token概率
    def get_output_embeddings(self):
        return self.lm_head
    # 设置新的语言模型头
    def set_output_embeddings(self, new_embeddings):
        self.lm_head = new_embeddings
    # 绑定词嵌入权重：
    #     如果配置允许（tie_word_embeddings=True），
    #     绑定编码器、解码器词嵌入权重与语言模型头权重为同一个张量，
    #     减少模型参数，提升共享一致性。
    def _tie_weights(self):
        if self.config.tie_word_embeddings:
            self.model._tie_weights()
            # self.lm_head被设置成self.model.shared的copy
            self._tie_or_clone_weights(self.lm_head, self.model.shared)

    @add_start_docstrings_to_model_forward(BART_INPUTS_DOCSTRING)
    @replace_return_docstrings(output_type=Seq2SeqLMOutput, config_class=_CONFIG_FOR_DOC)
    @add_end_docstrings(BART_GENERATION_EXAMPLE)
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, Seq2SeqLMOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
            config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
            (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.

        Returns:
        """
        # 在类 BartForConditionalGeneration 的 forward 方法上使用了 @replace_return_docstrings(...) 
        # 装饰器，但这个方法的 docstring 中 没有预留 Returns: 或 Return: 这一行作为占位符，
        # 而 replace_return_docstrings 要求这样一个占位符来替换文档。所以上面的文档字符串不能删
        # 是否返回字典结构
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        # 如果有传入标签的话
        if labels is not None:
            # 由于提供了“labels”，因此“use_cache”参数更改为“False”
            if use_cache: # 如果使用缓存
                logger.warning("The `use_cache` argument is changed to `False` since `labels` is provided.")
            use_cache = False # 训练时不需要用缓存k,v,因为训练时是并行的
            # labels存在且未传入decoder_input_ids，则自动右移生成decoder_input_ids
            if decoder_input_ids is None and decoder_inputs_embeds is None:
                # 这时设置目标序列输入为labels[:,:-1]
                decoder_input_ids = shift_tokens_right(
                    labels, self.config.pad_token_id, self.config.decoder_start_token_id
                )
        # 调用底层BART模型的forward，得到隐藏状态等信息
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            encoder_outputs=encoder_outputs,
            decoder_attention_mask=decoder_attention_mask,
            head_mask=head_mask,
            decoder_head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            decoder_inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        # 通过语言模型头将解码器最后隐藏层映射到词表大小，得到预测logits
        lm_logits = self.lm_head(outputs[0])
        # 加上偏置final_logits_bias，调整最终概率分布
        lm_logits = lm_logits + self.final_logits_bias.to(lm_logits.device)

        masked_lm_loss = None # 掩码语言损失
        if labels is not None: # 如果标签不是None
            labels = labels.to(lm_logits.device)
            # 计算交叉熵损失，只对非-100的token计算
            loss_fct = CrossEntropyLoss()
            # 计算交叉熵损失
            masked_lm_loss = loss_fct(lm_logits.view(-1, self.config.vocab_size), labels.view(-1))
        # 返回元组，包含loss(若计算)、logits和其他模型输出
        if not return_dict:
            output = (lm_logits,) + outputs[1:]
            return ((masked_lm_loss,) + output) if masked_lm_loss is not None else output
        # 返回结构化对象，方便访问每个组件的输出
        return Seq2SeqLMOutput(
            loss=masked_lm_loss,  # 若提供了标签，则此项为计算得到的语言建模损失；否则为 None 
            logits=lm_logits,  # 输出的 logits 形状为 [b,s,v]
            past_key_values=outputs.past_key_values, # 用于加速生成的缓存键值对
            decoder_hidden_states=outputs.decoder_hidden_states, # 解码器每一层的隐藏状态（可选输出）
            decoder_attentions=outputs.decoder_attentions,  # 解码器每一层的自注意力权重（可选输出）
            cross_attentions=outputs.cross_attentions, # 每层解码器对编码器的交叉注意力权重（可选输出）
            encoder_last_hidden_state=outputs.encoder_last_hidden_state, # 编码器最后一层输出的隐藏状态
            encoder_hidden_states=outputs.encoder_hidden_states, # 编码器每一层的隐藏状态（可选输出）
            encoder_attentions=outputs.encoder_attentions, # 编码器每一层的自注意力权重（可选输出）
        )
    # 根据标签生成decoder输入id（右移一位），
    # 用于训练时teacher forcing，保证预测时的输入合理。
    def prepare_decoder_input_ids_from_labels(self, labels: torch.Tensor):
        return shift_tokens_right(labels, self.config.pad_token_id, self.config.decoder_start_token_id)
    # 用于在 beam search 过程中，按照 beam 的重排序索引 `beam_idx` 对缓存的 past_key_values 重新排序，
    # 以确保后续解码步骤使用正确顺序的上下文信息
    @staticmethod  # 声明为静态方法，不依赖类实例状态
    def _reorder_cache(past_key_values, beam_idx):
        reordered_past = ()  # 用于收集重排后的每层缓存
        # layer_past 是一个元组，通常包含：
        # (self_attn_key, self_attn_value, cross_attn_key, cross_attn_value)
        # 其中 cross attention 部分在 beam search 时是静态不变的，不需要重排
        for layer_past in past_key_values:
            reordered_past += (
                tuple(past_state.index_select(0, beam_idx.to(past_state.device)) # 按 beam_idx 对 batch 维重排
                      for past_state in layer_past[:2])  # 只对 self-attn 的 key/value 进行重排
                + layer_past[2:],# cross-attn 的 key/value 保持原样
            )
        return reordered_past

In [ ]:
# help(BartForConditionalGeneration.forward)

In [82]:
from transformers import AutoTokenizer, BartForConditionalGeneration

In [83]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [84]:
# PG&E 表示，他们安排了停电措施，是为了应对在干燥天气中预测到的大风。其目的是为了降低野火风险。
# 预计将有近 80 万用户受到此次停电的影响，停电将至少持续到明天中午。
ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
    )

In [85]:
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors="pt")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [86]:
inputs

{'input_ids': tensor([[    0,  8332,   947,   717,  2305,    24,  1768,     5,   909,  4518,
            11,  1263,     7,  5876,    13,   239,  2372,  2876,  3841,  1274,
             4,    20,  4374,    16,     7,  1888,     5,   810,     9, 12584,
             4,  9221,  5735,  7673,   916,    58,  1768,     7,    28,  2132,
            30,     5,  2572, 10816,    61,    58,   421,     7,    94,   149,
            23,   513, 15372,  3859,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}

In [87]:
# 生成摘要
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=20)

In [88]:
summary_ids

tensor([[   2,    0, 8332,  947,  717, 1768,    5,  909, 4518,   11, 1263,    7,
         5876,   13,  239, 2372, 2876, 3841, 1274,    2]])

In [89]:
# 跳过特殊token
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# PG&E 因干燥天气中预测到的大风而安排了停电。

'PG&E scheduled the blackouts in response to forecasts for high winds amid dry conditions'

In [90]:
from transformers import AutoTokenizer, BartForConditionalGeneration

In [91]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [92]:
# 我朋友们是 <mask>，但他们吃了太多碳水。
TXT = "My friends are <mask> but they eat too many carbs."
input_ids = tokenizer([TXT], return_tensors="pt")["input_ids"]

In [93]:
input_ids

tensor([[    0,  2387,   964,    32, 50264,    53,    51,  3529,   350,   171,
         33237,     4,     2]])

In [94]:
logits = model(input_ids).logits

In [95]:
logits.shape

torch.Size([1, 13, 50265])

In [100]:
(input_ids[0] == tokenizer.mask_token_id).nonzero()

tensor([[4]])

In [96]:
masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()

In [97]:
masked_index # 掩码位置的索引

4

In [103]:
logits[0, masked_index].shape

torch.Size([50265])

In [104]:
probs = logits[0, masked_index].softmax(dim=0)

In [106]:
probs.topk(5)

torch.return_types.topk(
values=tensor([0.0929, 0.0917, 0.0855, 0.0579, 0.0412], grad_fn=<TopkBackward0>),
indices=tensor([  45,  205, 2245,  372,  182]))

In [105]:
values, predictions = probs.topk(5) # 返回最大的5个概率,还有对应的索引

In [107]:
tokenizer.decode(predictions)

' not good healthy great very'

In [108]:
# 这个“but”表示转折，“吃太多碳水”通常与“不健康”有关，因此前面很可能是说“他们是健康的”，但
# 后半句指出了一个与健康相矛盾的行为。因此，“healthy”作为对比最自然。
tokenizer.decode(predictions).split()

['not', 'good', 'healthy', 'great', 'very']

In [109]:
config.num_labels

3

In [110]:
# Bart模型，适用于序列分类任务（如GLUE），在 pooled output 上加了一个线性层作为分类头。
@add_start_docstrings(
    """
    Bart model with a sequence classification/head on top (a linear layer on top of the pooled output) e.g. for GLUE
    tasks.
    """,
    BART_START_DOCSTRING,
)
class BartForSequenceClassification(BartPreTrainedModel):
    # 指定权重共享的参数，encoder和decoder共享词嵌入表示
    _tied_weights_keys = ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight"]

    def __init__(self, config: BartConfig, **kwargs):
        super().__init__(config, **kwargs)
        self.model = BartModel(config) # 初始化主模型（包含 encoder + decoder）
        self.classification_head = BartClassificationHead( # 分类头：对encoder输出的句子表示进行分类
            config.d_model, # 输入维度：encoder输出维度
            config.d_model,   # 中间维度（可能用于非线性变换）
            config.num_labels,    # 输出类别数
            config.classifier_dropout, # dropout概率
        )

         # 初始化权重，包括 classification head 和模型内部参数
        self.post_init()

    @add_start_docstrings_to_model_forward(BART_INPUTS_DOCSTRING)
    @add_code_sample_docstrings(
        checkpoint=_CHECKPOINT_FOR_SEQUENCE_CLASSIFICATION,
        output_type=Seq2SeqSequenceClassifierOutput,
        config_class=_CONFIG_FOR_DOC,
        expected_output=_SEQ_CLASS_EXPECTED_OUTPUT,
        expected_loss=_SEQ_CLASS_EXPECTED_LOSS,
    )
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, Seq2SeqSequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        if labels is not None:
            use_cache = False  # 如果有标签，表示训练阶段，禁用缓存以节省显存
         # 当前不支持 inputs_embeds 输入方式
        if input_ids is None and inputs_embeds is not None:
            raise NotImplementedError(
                f"Passing input embeddings is currently not supported for {self.__class__.__name__}"
            )
        # 调用 BartModel 得到输出
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            head_mask=head_mask,
            decoder_head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            encoder_outputs=encoder_outputs,
            inputs_embeds=inputs_embeds,
            decoder_inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]  # encoder最后一层的输出 (b,s,d)
        # 定位每个样本中<eos> token的位置，用于抽取句子表示
        eos_mask = input_ids.eq(self.config.eos_token_id).to(hidden_states.device)
        # 每个样本必须有相同数量的eos标记，否则报错（模型设计依赖这个假设）
        if len(torch.unique_consecutive(eos_mask.sum(1))) > 1:
            raise ValueError("All examples must have the same number of <eos> tokens.")
        # 抽取每个样本中最后一个<eos> token对应的hidden state作为句子表示
        sentence_representation = hidden_states[eos_mask, :].view(
            hidden_states.size(0), -1, hidden_states.size(-1))[
            :, -1, :
        ]
        logits = self.classification_head(sentence_representation)  # 分类头得到 logits

        loss = None # 初始化损失
        if labels is not None: # 如果传入了labels
            labels = labels.to(logits.device)
            # 如果配置里没有设定问题类型
            if self.config.problem_type is None:
                if self.config.num_labels == 1: # 如果配置中的标签数为1,就设定问题类型为回归
                    self.config.problem_type = "regression"
                # 如果标签数>1 并且标签类型是整数型
                elif self.config.num_labels > 1 and (
                    labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification" # 设定为单标签分类
                else: # 其他情况 就是标签类型不是整数的情况
                    self.config.problem_type = "multi_label_classification" # 问题类型设定为多标签分类
            # 如果是回归问题
            if self.config.problem_type == "regression":
                loss_fct = MSELoss() # 设定损失函数为mse
                if self.config.num_labels == 1: # 如果标签类别是1
                    loss = loss_fct(logits.squeeze(), labels.squeeze()) # 计算损失
                else: # 如果是其他情况 不用squeeze
                    loss = loss_fct(logits, labels)
            # 如果是单标签分类问题
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss() # 损失为交叉熵损失
                loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1)) # 计算损失
            # 如果是多标签分类问题
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss() # 设定损失为二元交叉熵损失
                loss = loss_fct(logits, labels) # 计算损失
        if not return_dict: # 返回元组
            output = (logits,) + outputs[1:]
            return ((loss,) + output) if loss is not None else output
        # 返回结构化输出
        return Seq2SeqSequenceClassifierOutput(
            loss=loss, # 损失
            logits=logits, # 分类的logits分数
            past_key_values=outputs.past_key_values, # past_k_v
            decoder_hidden_states=outputs.decoder_hidden_states, 
            decoder_attentions=outputs.decoder_attentions,
            cross_attentions=outputs.cross_attentions,
            encoder_last_hidden_state=outputs.encoder_last_hidden_state, # 编码器输出
            encoder_hidden_states=outputs.encoder_hidden_states,
            encoder_attentions=outputs.encoder_attentions,
        )

In [ ]:
help(BartForSequenceClassification.forward)

In [112]:
# 单标签分类示例：
# import torch
from transformers import AutoTokenizer, BartForSequenceClassification

In [113]:
tokenizer = AutoTokenizer.from_pretrained("valhalla/bart-large-sst2")
model = BartForSequenceClassification.from_pretrained("valhalla/bart-large-sst2")

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [114]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt") # 你好，我的狗狗很可爱

In [115]:
inputs

{'input_ids': tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [117]:
print(config.problem_type,config.num_labels)

None 3


In [118]:
with torch.no_grad():
    logits = model(**inputs).logits

In [119]:
logits.shape

torch.Size([1, 2])

In [120]:
predicted_class_id = logits.argmax().item()

In [121]:
predicted_class_id

1

In [122]:
model.config.id2label[predicted_class_id]

'POSITIVE'

In [123]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [124]:
# 要在“num_labels”类上训练模型，您可以将“num_labels=num_labels”传递给“.from_pretrained(...)”
num_labels = len(model.config.id2label)

In [125]:
model = BartForSequenceClassification.from_pretrained("valhalla/bart-large-sst2", num_labels=num_labels)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


In [126]:
labels = torch.tensor([1])
loss = model(**inputs, labels=labels).loss

In [127]:
round(loss.item(), 2)

0.0

In [ ]:
# 多标签分类示例

In [128]:
tokenizer = AutoTokenizer.from_pretrained("valhalla/bart-large-sst2")
model = BartForSequenceClassification.from_pretrained(
    "valhalla/bart-large-sst2", problem_type="multi_label_classification") # 问题类型=多标签分类

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


In [129]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

In [130]:
inputs

{'input_ids': tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [131]:
with torch.no_grad():
    logits = model(**inputs).logits

In [132]:
logits

tensor([[-5.4279,  4.7800]])

In [133]:
torch.sigmoid(logits).squeeze(dim=0)

tensor([0.0044, 0.9917])

In [134]:
torch.arange(0, logits.shape[-1])

tensor([0, 1])

In [137]:
torch.sigmoid(logits).squeeze(dim=0) > 0.5

tensor([False,  True])

In [145]:
predicted_class_ids = (torch.sigmoid(logits).squeeze(dim=0) > 0.5).long()

In [146]:
predicted_class_ids

tensor([0, 1])

In [147]:
num_labels = len(model.config.id2label)

In [148]:
num_labels 

2

In [149]:
model = BartForSequenceClassification.from_pretrained(
        "valhalla/bart-large-sst2", num_labels=num_labels, problem_type="multi_label_classification"
    )
    

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


In [151]:
predicted_class_ids[None, :].clone()

tensor([[0, 1]])

In [152]:
torch.nn.functional.one_hot(predicted_class_ids[None, :].clone(), num_classes=num_labels)

tensor([[[1, 0],
         [0, 1]]])

In [154]:
labels = torch.sum(
       torch.nn.functional.one_hot(predicted_class_ids[None, :].clone(), num_classes=num_labels), dim=1
    ).to(torch.float)
labels

tensor([[1., 1.]])

In [155]:
loss = model(**inputs, labels=labels).loss

In [156]:
loss

tensor(2.7203, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [207]:
aa=torch.randn((2,5,2))

In [208]:
aa

tensor([[[-0.0324,  0.6286],
         [ 0.0170, -1.5568],
         [ 1.5358,  0.1274],
         [-0.7292, -0.2785],
         [ 0.4030, -0.1629]],

        [[-0.5587, -0.5935],
         [-0.0166, -1.4312],
         [ 0.5255,  1.5923],
         [ 1.3108, -0.2773],
         [-2.4679,  0.2038]]])

In [209]:
aa.split(2, dim=-1)  # 这个不管用,是一个长度为1的元组

(tensor([[[-0.0324,  0.6286],
          [ 0.0170, -1.5568],
          [ 1.5358,  0.1274],
          [-0.7292, -0.2785],
          [ 0.4030, -0.1629]],
 
         [[-0.5587, -0.5935],
          [-0.0166, -1.4312],
          [ 0.5255,  1.5923],
          [ 1.3108, -0.2773],
          [-2.4679,  0.2038]]]),)

In [210]:
len(aa.split(2, dim=-1))

1

In [211]:
aa.split(1, dim=-1)

(tensor([[[-0.0324],
          [ 0.0170],
          [ 1.5358],
          [-0.7292],
          [ 0.4030]],
 
         [[-0.5587],
          [-0.0166],
          [ 0.5255],
          [ 1.3108],
          [-2.4679]]]),
 tensor([[[ 0.6286],
          [-1.5568],
          [ 0.1274],
          [-0.2785],
          [-0.1629]],
 
         [[-0.5935],
          [-1.4312],
          [ 1.5923],
          [-0.2773],
          [ 0.2038]]]))

In [212]:
print(type(aa.split(1, dim=-1)),len(aa.split(1, dim=-1)))

<class 'tuple'> 2


In [213]:
(aa.split(1, dim=-1))[0].squeeze(-1).contiguous()

tensor([[-0.0324,  0.0170,  1.5358, -0.7292,  0.4030],
        [-0.5587, -0.0166,  0.5255,  1.3108, -2.4679]])

In [217]:
(aa.split(1, dim=-1))[0].shape

torch.Size([2, 5, 1])

In [218]:
aa = (aa.split(1, dim=-1))[0].squeeze(-1).contiguous()

In [219]:
aa.shape

torch.Size([2, 5])

In [221]:
aa.size(1)

5

In [224]:
@add_start_docstrings(
    """
    BART Model with a span classification head on top for extractive question-answering tasks like SQuAD (a linear
    layer on top of the hidden-states output to compute `span start logits` and `span end logits`).
    """,
    BART_START_DOCSTRING,
)
class BartForQuestionAnswering(BartPreTrainedModel):
    _tied_weights_keys = ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight"]

    def __init__(self, config):
        super().__init__(config)
        config.num_labels = 2 # 设置标签数为2，分别用于start位置和end位置
        self.num_labels = config.num_labels

        self.model = BartModel(config)   # 初始化底层的BART模型（encoder + decoder）
        # QA头部，用于预测start和end位置。输出形状为 (batch_size, seq_len, 2)
        self.qa_outputs = nn.Linear(config.hidden_size, config.num_labels)
        # 初始化所有模块参数
        self.post_init()

    @add_start_docstrings_to_model_forward(BART_INPUTS_DOCSTRING)
    @add_code_sample_docstrings(
        checkpoint=_CHECKPOINT_FOR_QA,
        output_type=Seq2SeqQuestionAnsweringModelOutput,
        config_class=_CONFIG_FOR_DOC,
        expected_loss=_QA_EXPECTED_LOSS,
        expected_output=_QA_EXPECTED_OUTPUT,
    )
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        start_positions: Optional[torch.LongTensor] = None,
        end_positions: Optional[torch.LongTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, Seq2SeqQuestionAnsweringModelOutput]:
        r"""
        start_positions (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for position (index) of the start of the labelled span for computing the token classification loss.
            Positions are clamped to the length of the sequence (*sequence_length*). Position outside of the sequence
            are not taken into account for computing the loss.
        end_positions (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for position (index) of the end of the labelled span for computing the token classification loss.
            Positions are clamped to the length of the sequence (*sequence_length*). Position outside of the sequence
            are not taken into account for computing the loss.
        """
        # 默认使用配置文件中的return_dict设定
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        # 如果提供了标签，关闭缓存机制（不保留KV缓存）
        if start_positions is not None and end_positions is not None:
            use_cache = False
        # 调用BART主模型，返回包含last_hidden_state在内的多个输出
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            head_mask=head_mask,
            decoder_head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            encoder_outputs=encoder_outputs,
            inputs_embeds=inputs_embeds,
            decoder_inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]  # 获取encoder输出的hidden states，形状为 (batch, seq_len, hidden_dim)
        # 线性层输出两个logits：start位置和end位置，形状为 (batch, seq_len, 2)
        logits = self.qa_outputs(sequence_output)
        # 拆分start和end的logits，并去掉最后一个维度，结果形状为 (batch, seq_len)
        # 这个拆分后,左侧是start_logits,右侧是end_logits 
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous() # 紧凑维度
        end_logits = end_logits.squeeze(-1).contiguous()

        total_loss = None # 问答的损失
        # 如果起始和结束位置都不是None
        if start_positions is not None and end_positions is not None:
             # 如果标签维度是 (batch, 1)，先squeeze成 (batch,)
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            # 获取模型输出的最大序列长度，用于后续裁剪非法标签位置
            ignored_index = start_logits.size(1)
            # 将标签位置限制在 [0, 序列长度] 范围内
            # 设计意图：部分数据集中可能存在越界标签，这里使用 clamp 保证标签合法，防止计算损失时报错
            start_positions = start_positions.clamp(0, ignored_index)
            end_positions = end_positions.clamp(0, ignored_index)
            # 定义交叉熵损失函数，并忽略非法的标签位置（被置为 ignored_index）
            # 设计意图：模型输出的是每个 token 的 logits，需要用 token 索引作为 target 计算分类损失
            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
             # 分别计算起始位置和结束位置的分类损失
            start_loss = loss_fct(start_logits, start_positions)  # 答案起始位置的预测损失
            end_loss = loss_fct(end_logits, end_positions) # 答案结束位置的预测损失
            # 最终损失是起始位置与结束位置损失的平均
            # 设计意图：起始与结束同等重要，平均损失作为整体优化目标
            total_loss = (start_loss + end_loss) / 2 
        # 返回元组
        if not return_dict:
            output = (
                start_logits,
                end_logits,
            ) + outputs[1:]
            return ((total_loss,) + output) if total_loss is not None else output
        # 返回结构化输出
        return Seq2SeqQuestionAnsweringModelOutput(
            loss=total_loss, # 损失
            start_logits=start_logits, # 答案开始的logits
            end_logits=end_logits,  # 答案结束的logits
            past_key_values=outputs.past_key_values,
            decoder_hidden_states=outputs.decoder_hidden_states,
            decoder_attentions=outputs.decoder_attentions,
            cross_attentions=outputs.cross_attentions,
            encoder_last_hidden_state=outputs.encoder_last_hidden_state,
            encoder_hidden_states=outputs.encoder_hidden_states,
            encoder_attentions=outputs.encoder_attentions,
        )

In [ ]:
help(BartForQuestionAnswering.forward)

In [226]:
from transformers import AutoTokenizer, BartForQuestionAnswering

In [227]:
tokenizer = AutoTokenizer.from_pretrained("valhalla/bart-large-finetuned-squadv1")
model = BartForQuestionAnswering.from_pretrained("valhalla/bart-large-finetuned-squadv1")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels will be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [228]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

In [229]:
inputs = tokenizer(question, text, return_tensors="pt")

In [230]:
inputs

{'input_ids': tensor([[    0, 12375,    21,  2488,   289, 13919,   116,     2,     2, 24021,
           289, 13919,    21,    10,  2579, 29771,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [231]:
with torch.no_grad():
  outputs = model(**inputs)

In [233]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [234]:
print(answer_start_index,answer_end_index)

tensor(14) tensor(15)


In [235]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]

In [236]:
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

' nice puppet'

In [237]:
# target is "nice puppet"
target_start_index = torch.tensor([14])
target_end_index = torch.tensor([15])

In [238]:
outputs = model(
    **inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss
round(loss.item(), 2)

0.59

In [239]:
# 该包装类用于在与 EncoderDecoderModel 框架结合使用时，正确加载 BART 的解码器权重。
# 设计意图是将 BART 解码器封装为一个独立模块，使其能被统一接口调用并加载预训练检查点。
class BartDecoderWrapper(BartPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        # 初始化 BART 解码器组件，传入配置对象
        # 此处不包括编码器部分，仅构造解码器，便于与其他编码器组合使用
        self.decoder = BartDecoder(config)
    # 将所有输入直接传给解码器的前向函数，确保与 BARTDecoder 接口一致
    def forward(self, *args, **kwargs):
        return self.decoder(*args, **kwargs)

In [263]:
# BART 解码器 + 语言建模头（lm_head），用于自回归文本生成任务。lm_head 权重与输入嵌入权重共享。
@add_start_docstrings(
    """
    BART decoder with a language modeling head on top (linear layer with weights tied to the input embeddings).
    """,
    BART_START_DOCSTRING,
)
class BartForCausalLM(BartPreTrainedModel, GenerationMixin):
    _tied_weights_keys = ["lm_head.weight"]  # 指定哪些权重需要与其它模块（如嵌入层）共享

    def __init__(self, config):
        config = copy.deepcopy(config)
        config.is_decoder = True # 明确当前模型只作为 decoder 使用
        config.is_encoder_decoder = False # 禁止使用 encoder-decoder 模式
        super().__init__(config)
        # 包装解码器结构，隐藏底层 decoder 实现细节
        self.model = BartDecoderWrapper(config)
         # 输出层：将 decoder 的隐藏状态映射为词表 logits
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        # 初始化所有模块参数
        self.post_init()
     # 获取输入嵌入层（与 decoder 的嵌入层一致）
    def get_input_embeddings(self):
        return self.model.decoder.embed_tokens
     # 设置 decoder 的嵌入层（用于 weight tying）
    def set_input_embeddings(self, value):
        self.model.decoder.embed_tokens = value
    
    def get_output_embeddings(self):
        return self.lm_head

    def set_output_embeddings(self, new_embeddings):
        self.lm_head = new_embeddings
    # 设置解码器
    def set_decoder(self, decoder):
        self.model.decoder = decoder
    # 获取解码器
    def get_decoder(self):
        return self.model.decoder

    @replace_return_docstrings(output_type=CausalLMOutputWithCrossAttentions, config_class=_CONFIG_FOR_DOC)
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        encoder_hidden_states: Optional[torch.FloatTensor] = None,
        encoder_attention_mask: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, CausalLMOutputWithCrossAttentions]:
        r"""
        Args:
            input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
                Indices of input sequence tokens in the vocabulary. Padding will be ignored by default should you
                provide it.

                Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.encode`] and
                [`PreTrainedTokenizer.__call__`] for details.

                [What are input IDs?](../glossary#input-ids)
            attention_mask (`torch.Tensor` of shape `(batch_size, sequence_length)`, *optional*):
                Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:

                - 1 for tokens that are **not masked**,
                - 0 for tokens that are **masked**.

                [What are attention masks?](../glossary#attention-mask)
            encoder_hidden_states  (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
                Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention
                if the model is configured as a decoder.
            encoder_attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
                Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used
                in the cross-attention if the model is configured as a decoder. Mask values selected in `[0, 1]`:
            head_mask (`torch.Tensor` of shape `(decoder_layers, decoder_attention_heads)`, *optional*):
                Mask to nullify selected heads of the attention modules. Mask values selected in `[0, 1]`:

                - 1 indicates the head is **not masked**,
                - 0 indicates the head is **masked**.

            cross_attn_head_mask (`torch.Tensor` of shape `(decoder_layers, decoder_attention_heads)`, *optional*):
                Mask to nullify selected heads of the cross-attention modules. Mask values selected in `[0, 1]`:

                - 1 indicates the head is **not masked**,
                - 0 indicates the head is **masked**.

            past_key_values (`tuple(tuple(torch.FloatTensor))`, *optional*, returned when `use_cache=True` is passed or when `config.use_cache=True`):
                Tuple of `tuple(torch.FloatTensor)` of length `config.n_layers`, with each tuple having 2 tensors of
                shape `(batch_size, num_heads, sequence_length, embed_size_per_head)`) and 2 additional tensors of
                shape `(batch_size, num_heads, encoder_sequence_length, embed_size_per_head)`. The two additional
                tensors are only required when the model is used as a decoder in a Sequence to Sequence model.

                Contains pre-computed hidden-states (key and values in the self-attention blocks and in the
                cross-attention blocks) that can be used (see `past_key_values` input) to speed up sequential decoding.

                If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those
                that don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of
                all `decoder_input_ids` of shape `(batch_size, sequence_length)`.
            labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
                Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
                config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
                (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.
            use_cache (`bool`, *optional*):
                If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding
                (see `past_key_values`).

                - 1 for tokens that are **not masked**,
                - 0 for tokens that are **masked**.
            output_attentions (`bool`, *optional*):
                Whether or not to return the attentions tensors of all attention layers. See `attentions` under
                returned tensors for more detail.
            output_hidden_states (`bool`, *optional*):
                Whether or not to return the hidden states of all layers. See `hidden_states` under returned tensors
                for more detail.
            return_dict (`bool`, *optional*):
                Whether or not to return a [`~utils.ModelOutput`] instead of a plain tuple.

        Returns:

        Example:

        ```python
        >>> from transformers import AutoTokenizer, BartForCausalLM

        >>> tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
        >>> model = BartForCausalLM.from_pretrained("facebook/bart-base", add_cross_attention=False)
        >>> assert model.config.is_decoder, f"{model.__class__} has to be configured as a decoder."
        >>> inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
        >>> outputs = model(**inputs)

        >>> logits = outputs.logits
        >>> expected_shape = [1, inputs.input_ids.shape[-1], model.config.vocab_size]
        >>> list(logits.shape) == expected_shape
        True
        ```"""
        # 如果未显式指定，使用 config 中默认值
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # 解码器输出由（dec_features、layer_state、dec_hidden、dec_attn）组成
        # 解码器前向传播，返回 decoder 的输出（包含隐藏状态、注意力等）
        outputs = self.model.decoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            head_mask=head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        # 使用语言模型头将隐藏状态映射为 logits（词表维度）
        logits = self.lm_head(outputs[0])

        loss = None # 初始损失
        if labels is not None:
            labels = labels.to(logits.device)
            loss_fct = CrossEntropyLoss() # 设置损失为交叉熵
            # 将 logits 和 labels 展平后计算损失（忽略 label=-100 的位置）
            loss = loss_fct(logits.view(-1, self.config.vocab_size), labels.view(-1))
        # 根据 return_dict 决定返回格式
        if not return_dict:
            output = (logits,) + outputs[1:]
            return (loss,) + output if loss is not None else output
        # 返回带有结构化字段的输出（包含 logits, loss, past key values 等）
        return CausalLMOutputWithCrossAttentions(
            loss=loss, # 损失
            logits=logits, # 下个token的概率分布
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            cross_attentions=outputs.cross_attentions,
        )

    @staticmethod # 静态方法
    def _reorder_cache(past_key_values, beam_idx):
        # 在 beam search 解码过程中重排序 past_key_values
        # 每层的 past_state 维度为 (batch, num_heads, seq_len, head_dim)，根据 beam_idx 选择新的顺序
        reordered_past = ()
        for layer_past in past_key_values:
            reordered_past += (
                tuple(past_state.index_select(0, beam_idx.to(past_state.device)) for past_state in layer_past),
            )
        return reordered_past

In [ ]:
help(BartForCausalLM.forward)

In [242]:
from transformers import AutoTokenizer

In [243]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
# model = BartForCausalLM.from_pretrained("facebook/bart-base", add_cross_attention=False)

In [262]:
model=BartForCausalLM(config)

In [246]:
model.config.is_decoder

True

In [247]:
model.__class__

__main__.BartForCausalLM

In [248]:
# 断言
assert model.config.is_decoder, f"{model.__class__} has to be configured as a decoder."

In [249]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

In [250]:
inputs

{'input_ids': tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [251]:
outputs = model(**inputs)

In [253]:
outputs.logits.shape

torch.Size([1, 8, 50265])

In [254]:
logits = outputs.logits

In [256]:
logits.shape

torch.Size([1, 8, 50265])

In [257]:
inputs.input_ids.shape[-1]

8

In [258]:
model.config.vocab_size

50265

In [259]:
expected_shape = [1, inputs.input_ids.shape[-1], model.config.vocab_size] # 预期形状

In [260]:
list(logits.shape) == expected_shape

True

In [ ]:
__all__ = [
    "BartForCausalLM",
    "BartForConditionalGeneration",
    "BartForQuestionAnswering",
    "BartForSequenceClassification",
    "BartModel",
    "BartPreTrainedModel",
    "BartPretrainedModel",
    "PretrainedBartModel",
]